Iran Twitter / X Data Analysis Project

In [ ]:
# ====================================================
# Iran Twitter / X Data Analysis Project
# ====================================================
# Course: Data Science Final Project (SCE)

In [ ]:
# Install required packages
%pip install -q requests pandas urllib3 emoji transformers sentence-transformers scikit-learn matplotlib seaborn plotly numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# **italicized text*POI Scraper (Wikipedia Categories → CSV)**

In [ ]:
# ============================================
# File: step1_collect_wikipedia_categories.py (UPDATED)
#
# Description:
#   Recursively collects people/pages from Wikipedia categories (including subcategories),
#   and saves a CSV per category with columns: Name, Wikipedia_Link.
#   Output structure (per course spec):
#       <IRAN_DIR>/
#         POIs/
#           <slug>/
#             <slug>_wikipedia.csv
#   Also produces a summary table and a ZIP with all CSVs.
# ============================================

import requests, time, os, zipfile
import pandas as pd
from urllib.parse import quote
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# ---------------- IRAN_DIR resolution ----------------
# Set the Iran directory path (adjust if needed)
IRAN_DIR = r"C:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran"

if not os.path.isdir(IRAN_DIR):
    raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran' בנתיב: {IRAN_DIR}")

# Base POIs directory (as per course structure)
POIS_DIR = os.path.join(IRAN_DIR, "POIs")
os.makedirs(POIS_DIR, exist_ok=True)

# ---------------- API config ----------------
BASE_API = "https://en.wikipedia.org/w/api.php"
HEADERS = {"User-Agent": "SCE-DataScience-FinalProject/1.0 (contact: student@example.com)"}

session = requests.Session()
retries = Retry(total=5, backoff_factor=0.8, status_forcelist=[403,429,500,502,503,504])
session.mount("https://", HTTPAdapter(max_retries=retries))

def api_get(params):
    """Thin wrapper around MediaWiki API GET with a fallback UA on 403."""
    r = session.get(BASE_API, params=params, headers=HEADERS, timeout=30)
    if r.status_code == 403:
        time.sleep(1)
        r = session.get(BASE_API, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    return r.json()

def get_category_members(category_title, cmtype="page"):
    """
    Fetches up to all members of a category, paging through 'continue'.
    cmtype can be "page" (actual pages) or "subcat" (subcategories).
    """
    members, params = [], {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": category_title,
        "cmlimit": "500",
        "cmtype": cmtype,
        "format": "json"
    }
    while True:
        data = api_get(params)
        members.extend(data.get("query", {}).get("categorymembers", []))
        if "continue" in data:
            params["cmcontinue"] = data["continue"]["cmcontinue"]
        else:
            break
    return members

def collect_people_from_category(root_category):
    """
    Breadth-first traversal from a root category:
      - collects 'page' members as rows {Name, Wikipedia_Link}
      - enqueues 'subcat' members for further traversal
    Returns a deduplicated DataFrame by Name.
    """
    seen, pages, queue = set(), [], [root_category]
    while queue:
        cat = queue.pop(0)
        if cat in seen:
            continue
        seen.add(cat)
        print(f"📂 סורק: {cat}")
        # pages
        for m in get_category_members(cat, "page"):
            title = m["title"]
            link  = "https://en.wikipedia.org/wiki/" + quote(title.replace(" ", "_"))
            pages.append({"Name": title, "Wikipedia_Link": link})
        # subcategories
        for sc in get_category_members(cat, "subcat"):
            queue.append(sc["title"])
        time.sleep(0.15)  # be polite
    return pd.DataFrame(pages).drop_duplicates(subset=["Name"]).reset_index(drop=True)

def safe_slug(cat):
    """
    Converts a category title to a folder/filename-safe slug.
    Example:
      "Category:Iranian physicians" -> "iranian_physicians"
    """
    name = cat.replace("Category:", "").strip()
    name = name.replace(" ", "_")
    for bad in ['"', "'", "'", """, """, "(", ")", "/", "\\", ":", "*", "?", "<", ">", "|", ",", ";", "—", "–"]:
        name = name.replace(bad, "")
    # compress consecutive underscores
    while "__" in name:
        name = name.replace("__", "_")
    return name.lower()

# ---------------- Categories to collect (expanded incl. healthcare) ----------------
categories = [
    # Existing:
    "Category:Government_ministers_of_Iran",
    "Category:Presidents_of_Iran",
    "Category:Vice_presidents_of_Iran",
    "Category:Iranian_ayatollahs",
    "Category:Iranian_actors",
    "Category:Iranian_singers",
    "Category:Iranian_scientists",
    "Category:Iranian_economists",
    "Category:Iranian_writers",
    "Category:Iranian_football_managers",

    # NEW — Healthcare related:
    "Category:Hospitals_in_Iran",
    "Category:Private_hospitals_in_Iran",
    "Category:Teaching_hospitals_in_Iran",
    "Category:Iranian_physicians",
    "Category:Iranian_cardiologists",
    "Category:Iranian_women_physicians",
    "Category:21st-century_Iranian_physicians",
    "Category:19th-century_Iranian_physicians",
    "Category:Medical_and_health_organisations_based_in_Iran",
    "Category:Healthcare_in_Iran",
    "Category:Medicine_in_Iran",
]

# ---------------- Main loop: collect, save per-spec, summarize ----------------
csv_files = []
summary_rows = []

for cat in categories:
    try:
        df = collect_people_from_category(cat)
        n = len(df)
        slug = safe_slug(cat)
        cat_dir = os.path.join(POIS_DIR, slug)
        os.makedirs(cat_dir, exist_ok=True)

        if n == 0:
            print(f"⚠️ אין נתונים עבור {cat}")
            summary_rows.append({"Category": cat, "Slug": slug, "SavedRows": 0, "CSV": None})
            continue

        csv_name = f"{slug}_wikipedia.csv"
        csv_path = os.path.join(cat_dir, csv_name)
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        csv_files.append(csv_path)

        print(f"✅ נשמר: {csv_path} | רשומות: {n}")
        summary_rows.append({"Category": cat, "Slug": slug, "SavedRows": n, "CSV": csv_path})

    except Exception as e:
        print(f"❌ שגיאה בקטגוריה {cat}: {e}")
        summary_rows.append({"Category": cat, "Slug": safe_slug(cat), "SavedRows": 0, "CSV": None})

# --- Summary table ---
summary_df = pd.DataFrame(summary_rows).sort_values("SavedRows", ascending=False).reset_index(drop=True)
total_saved = summary_df["SavedRows"].sum()

print("\n================= Summary =================")
for _, r in summary_df.iterrows():
    base = os.path.basename(r['CSV']) if (pd.notna(r['CSV']) and r['CSV']) else "-"
    print(f"• {r['Category']} -> {r['Slug']}: {r['SavedRows']} רשומות  |  קובץ: {base}")
print(f"\n📊 Total across files: {total_saved} rows")
print("===========================================\n")

# Display summary table
display(summary_df)

# ---------------- ZIP all CSVs ----------------
if csv_files:
    zip_path = os.path.join(IRAN_DIR, "Iran_POIs_Wikipedia_Categories.zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in csv_files:
            # Save relative path in zip
            arcname = os.path.relpath(file, start=IRAN_DIR)
            zipf.write(file, arcname)
    print(f"💾 Created ZIP with all categories: {zip_path}")
else:
    print("ℹ️ No CSV files were created → no ZIP.")

ModuleNotFoundError: No module named 'requests'

# **Step 3: Wikidata Enrichment for ALL POI Folders**

In [ ]:
# ============================================
# Step 3: Wikidata Enrichment for ALL POI Folders (UPDATED)
# ============================================
# For each subfolder under POIs:
#  1) Locate *_wikipedia.csv (or the first .csv)
#  2) Detect the Wikipedia link column (fallback to Name)
#  3) Resolve wikidata_qid (+ wikidata_url)
#  4) Fetch details via SPARQL in batches
#  5) Save:
#     a) *_with_wikidata_ids_and_links.csv
#     b) *_with_wikidata_ids_and_links_wikidata_detailed.csv
# ============================================

import os, re, time, glob, json, math, random
import pandas as pd
import requests
from google.colab import drive

# ---------------- Drive mount & ROOT ----------------
drive.mount('/content/drive', force_remount=False)

def find_shared_folder(folder_name: str):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
assert BASE is not None, "❌ לא נמצאה תיקיית 'Iran' בדרייב. ודא שהוספת אותה ל-MyDrive/Shared Drive."
ROOT_DIR = os.path.join(BASE, "POIs")
os.makedirs(ROOT_DIR, exist_ok=True)
print(f"📁 ROOT_DIR: {ROOT_DIR}")

# ---------------- HTTP / Endpoints ----------------
UA = {"User-Agent": "SCE-DS-FinalProject/1.0 (contact: student@example.com)"}
MEDIAWIKI_API = "https://en.wikipedia.org/w/api.php"
SPARQL = "https://query.wikidata.org/sparql"

# ---------------- Helpers: retries ----------------
def http_get(url, params=None, headers=None, timeout=30, max_tries=4, sleep_base=0.8):
    """
    Simple GET with retries/backoff. Jitters a bit to be polite.
    """
    headers = headers or UA
    for attempt in range(1, max_tries + 1):
        try:
            r = requests.get(url, params=params, headers=headers, timeout=timeout)
            if r.status_code == 403 and headers is UA:
                # fallback UA
                r = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == max_tries:
                # last attempt: raise
                raise
            sleep_s = sleep_base * attempt + random.uniform(0, 0.3)
            time.sleep(sleep_s)
    # Shouldn't reach here
    raise RuntimeError("GET retries exhausted")

# ---------------- Column detection ----------------
def find_wikipedia_column(df: pd.DataFrame) -> str:
    # First pass: headers with hints
    for name in df.columns:
        n = str(name).strip().lower()
        if any(k in n for k in ["wikipedia", "wiki", "link", "url"]):
            return name
    # Second pass: sample content
    for name in df.columns:
        sample = " ".join(map(str, df[name].dropna().astype(str).head(20).tolist())).lower()
        if "wikipedia.org" in sample or sample.startswith("http"):
            return name
    # Fallback
    if "Wikipedia_Link" in df.columns:
        return "Wikipedia_Link"
    # As a last resort: if Name exists, we'll construct enwiki URLs from it
    if "Name" in df.columns:
        return None  # signal to construct from Name
    raise ValueError("לא מצאתי עמודת קישור לוויקיפדיה, ואין גם עמודת 'Name' לבניית קישורים.")

def name_to_enwiki_url(name: str) -> str:
    from urllib.parse import quote
    if not isinstance(name, str) or not name.strip():
        return None
    title = name.strip().replace(" ", "_")
    return f"https://en.wikipedia.org/wiki/{quote(title)}"

def wikipedia_url_to_title(url: str) -> str | None:
    if not isinstance(url, str) or not url:
        return None
    try:
        url = url.split("?")[0].split("#")[0]
        title = url.rstrip("/").split("/")[-1]
        return title if title else None
    except Exception:
        return None

# ---------------- QID resolution (with cache) ----------------
qid_cache = {}

def get_qid_from_wikipedia_url(url: str) -> str | None:
    """
    Resolve QID for a single Wikipedia page using the pageprops endpoint.
    """
    title = wikipedia_url_to_title(url)
    if not title:
        return None
    if title in qid_cache:
        return qid_cache[title]
    params = {
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "redirects": 1,
        "titles": title,
    }
    try:
        r = http_get(MEDIAWIKI_API, params=params, headers=UA, timeout=25)
        data = r.json()
        page = next(iter(data["query"]["pages"].values()))
        qid = page.get("pageprops", {}).get("wikibase_item")
        if qid:
            qid_cache[title] = qid
        return qid
    except Exception:
        return None

# ---------------- SPARQL batch fetch ----------------
def batch_fetch_wikidata_details(qids: list[str], batch_size: int = 50) -> dict:
    """
    Fetch properties for many QIDs via SPARQL using VALUES batching.
    Returns dict: { QID: {fields...}, ... }
    """
    results = {}
    # Normalize & unique
    qids = [q for q in qids if isinstance(q, str) and q.strip()]
    uniq = sorted(set(qids))
    if not uniq:
        return results

    def run_batch(subset):
        values = " ".join(f"wd:{q}" for q in subset)
        query = f"""
        SELECT ?item ?genderLabel ?occupationLabel ?countryLabel ?placeOfBirthLabel ?dateOfBirth WHERE {{
          VALUES ?item {{ {values} }}
          OPTIONAL {{ ?item wdt:P21 ?gender. }}
          OPTIONAL {{ ?item wdt:P106 ?occupation. }}
          OPTIONAL {{ ?item wdt:P27 ?country. }}
          OPTIONAL {{ ?item wdt:P19 ?placeOfBirth. }}
          OPTIONAL {{ ?item wdt:P569 ?dateOfBirth. }}
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,fa,en-gb". }}
        }}
        """
        r = http_get(SPARQL, params={"query": query, "format": "json"}, headers=UA, timeout=45)
        rows = r.json().get("results", {}).get("bindings", [])
        # collect multiple rows per item
        tmp = {}
        for b in rows:
            uri = b.get("item", {}).get("value", "")
            q = uri.rsplit("/", 1)[-1] if uri else None
            if not q:
                continue
            cur = tmp.setdefault(q, {"wikidata_gender": set(),
                                     "wikidata_occupation": set(),
                                     "wikidata_country_of_citizenship": set(),
                                     "wikidata_place_of_birth": None,
                                     "wikidata_date_of_birth": None})
            if "genderLabel" in b:
                cur["wikidata_gender"].add(b["genderLabel"]["value"])
            if "occupationLabel" in b:
                cur["wikidata_occupation"].add(b["occupationLabel"]["value"])
            if "countryLabel" in b:
                cur["wikidata_country_of_citizenship"].add(b["countryLabel"]["value"])
            if "placeOfBirthLabel" in b and not cur["wikidata_place_of_birth"]:
                cur["wikidata_place_of_birth"] = b["placeOfBirthLabel"]["value"]
            if "dateOfBirth" in b and not cur["wikidata_date_of_birth"]:
                cur["wikidata_date_of_birth"] = b["dateOfBirth"]["value"]
        # flatten sets
        for q, d in tmp.items():
            results[q] = {
                "wikidata_gender": "; ".join(sorted(d["wikidata_gender"])) or None,
                "wikidata_occupation": "; ".join(sorted(d["wikidata_occupation"])) or None,
                "wikidata_country_of_citizenship": "; ".join(sorted(d["wikidata_country_of_citizenship"])) or None,
                "wikidata_place_of_birth": d["wikidata_place_of_birth"],
                "wikidata_date_of_birth": d["wikidata_date_of_birth"],
            }

    for i in range(0, len(uniq), batch_size):
        subset = uniq[i:i+batch_size]
        # polite pacing
        try:
            run_batch(subset)
        except Exception as e:
            print(f"⚠️ SPARQL batch failed ({subset[0]}..): {e}. מנסה שוב בקבוצות קטנות יותר...")
            # fallback: try half batch to circumvent transient errors
            mid = len(subset)//2 or 1
            for chunk in (subset[:mid], subset[mid:]):
                try:
                    run_batch(chunk)
                except Exception as ee:
                    print(f"❌ SPARQL sub-batch failed ({chunk[0]}..): {ee}")
        time.sleep(0.5)

    return results

# ---------------- Main processing ----------------
folders = [f for f in sorted(os.listdir(ROOT_DIR)) if os.path.isdir(os.path.join(ROOT_DIR, f))]
print(f"🗂️ נמצאו {len(folders)} תיקיות POI לעיבוד.")

for folder in folders:
    FOLDER_PATH = os.path.join(ROOT_DIR, folder)
    print(f"\n📂 מעבד תיקייה: {folder}")

    # Locate input CSV (prefer *_wikipedia.csv)
    candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*_wikipedia.csv")))
    if not candidates:
        candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*.csv")))
    if not candidates:
        print("⚠️ לא נמצא קובץ CSV בתיקייה הזו, מדלג.")
        continue

    INPUT_CSV = candidates[0]
    try:
        df = pd.read_csv(INPUT_CSV)
    except Exception as e:
        print(f"❌ לא הצלחתי לקרוא את הקובץ {os.path.basename(INPUT_CSV)}: {e} — מדלג.")
        continue

    # Detect / construct Wikipedia links
    try:
        wiki_col = find_wikipedia_column(df)
    except Exception as e:
        print(f"⚠️ {e} — מנסה לבנות קישורי ויקיפדיה מ־'Name'...")
        wiki_col = None

    if wiki_col is None:
        if "Name" in df.columns:
            df["Wikipedia_Link"] = df["Name"].apply(name_to_enwiki_url)
            wiki_col = "Wikipedia_Link"
        else:
            print("❌ אין עמודת קישור/Name — מדלג על התיקייה.")
            continue

    print("🧩 עמודת ויקיפדיה:", wiki_col)

    # ---- A) Resolve QIDs (with simple in-memory cache) ----
    qids = []
    for url in df[wiki_col].fillna(""):
        qids.append(get_qid_from_wikipedia_url(url))
        time.sleep(0.08)  # polite throttle

    df["wikidata_qid"] = qids
    df["wikidata_url"] = df["wikidata_qid"].apply(lambda q: f"https://www.wikidata.org/wiki/{q}" if isinstance(q, str) and q else None)

    # Save mid file
    mid_path = os.path.join(
        FOLDER_PATH,
        re.sub(r"\.csv$", "", os.path.basename(INPUT_CSV)) + "_with_wikidata_ids_and_links.csv"
    )
    try:
        df.to_csv(mid_path, index=False, encoding="utf-8-sig")
        print(f"💾 נשמר ביניים: {os.path.basename(mid_path)}  (שורות: {len(df)})")
    except Exception as e:
        print(f"❌ שגיאה בשמירת קובץ ביניים: {e}")

    # ---- B) Fetch details via SPARQL in batches ----
    uniq_qids = [q for q in pd.Series(df["wikidata_qid"]).dropna().astype(str).unique().tolist() if q]
    details_map = batch_fetch_wikidata_details(uniq_qids, batch_size=50)

    details_rows = []
    for q in df["wikidata_qid"]:
        if isinstance(q, str) and q in details_map:
            details_rows.append(details_map[q])
        else:
            details_rows.append({
                "wikidata_gender": None,
                "wikidata_occupation": None,
                "wikidata_country_of_citizenship": None,
                "wikidata_place_of_birth": None,
                "wikidata_date_of_birth": None,
            })

    details_df = pd.DataFrame(details_rows)
    enriched = pd.concat([df, details_df], axis=1)

    out_path = os.path.join(
        FOLDER_PATH,
        re.sub(r"\.csv$", "", os.path.basename(INPUT_CSV)) + "_with_wikidata_ids_and_links_wikidata_detailed.csv"
    )
    try:
        enriched.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"✅ נשמר פלט סופי: {os.path.basename(out_path)}  (שורות: {len(enriched)})")
    except Exception as e:
        print(f"❌ שגיאה בשמירת הפלט הסופי: {e}")

print("\n🎉 הושלם עיבוד לכל התיקיות!")


# Step 4: Find Twitter handles for ALL POI folders

In [ ]:
# ============================================
# Step 4: Find Twitter handles for ALL POI folders
# Adds only: Twitter_username + Twitter_url (no twitter_source)
# ============================================

from google.colab import drive
import os, re, time, json, glob, requests, pandas as pd, random
from urllib.parse import urlparse, unquote
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---- DRIVE MOUNT ----
drive.mount('/content/drive', force_remount=False)

# ---- Locate shared 'Iran' folder dynamically ----
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

# אפשרות לעקוף ידנית עם משתנה סביבה (למי שרוצה):
BASE = os.environ.get('IRAN_DIR')
if not BASE or not os.path.isdir(BASE):
    BASE = find_shared_folder('Iran')

assert BASE and os.path.isdir(BASE), (
    "❌ לא נמצאה תיקיית 'Iran'. ודא שהוספת אותה ל-MyDrive/Shared Drive שלך. "
    "ניתן גם להגדיר ידנית: os.environ['IRAN_DIR'] = '/content/drive/…/Iran'"
)

# ירוץ על כל התיקיות תחת POIs
REL_ROOT = "POIs"
ROOT = os.path.join(BASE, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
print("📁 ROOT:", ROOT)

# ---------- HTTP session with retries ----------
UA = {"User-Agent": "SCE-DS-FinalProject/1.0 (contact: student@example.com)"}
session = requests.Session()
session.headers.update(UA)

# urllib3 v2: allowed_methods (not method_whitelist). עדיף frozenset
retry_cfg = Retry(
    total=5,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset({"GET"}),
)
session.mount("https://", HTTPAdapter(max_retries=retry_cfg))
session.mount("http://", HTTPAdapter(max_retries=retry_cfg))

SPARQL = "https://query.wikidata.org/sparql"
MEDIAWIKI_API = "https://en.wikipedia.org/w/api.php"

# ---------- helpers ----------
def pick_csv(folder):
    for pat in ["*_with_wikidata_ids_and_links_wikidata_detailed*.csv",
                "*_with_wikidata_ids_and_links*.csv",
                "*_wikipedia.csv",
                "*.csv"]:
        cand = sorted(glob.glob(os.path.join(folder, pat)))
        if cand: return cand[0]
    return None

def clean_handle(h):
    if not isinstance(h, str):
        return None
    h = h.strip().lstrip("@")
    h = re.sub(r"[/?#].*$", "", h)
    m = re.match(r"^[A-Za-z0-9_]{1,15}$", h)
    return m.group(0) if m else None

def extract_handle_from_url(url):
    """
    מחלץ ידית מכתובת twitter/x אם קיימת. לינקים מסוג /i/user/12345 לא מכילים ידית => נחזיר None.
    """
    try:
        u = url.split("?")[0].split("#")[0]
        host = urlparse(u).netloc.lower()
        if any(d in host for d in ["twitter.com","x.com","mobile.twitter.com","www.twitter.com","www.x.com"]):
            parts = urlparse(u).path.strip("/").split("/")
            if parts and parts[0].lower() not in {"i","intent","share","home"}:
                return clean_handle(parts[0])
    except:
        pass
    return None

def build_twitter_url(handle):
    if not isinstance(handle, str) or not handle.strip():
        return None
    return f"https://x.com/{handle.strip().lstrip('@')}"

def find_wikipedia_column(df):
    # ראשית: לפי שם עמודה
    for c in df.columns:
        n = str(c).lower()
        if any(k in n for k in ["wikipedia", "wiki", "link", "url"]):
            return c
    # שנית: לפי תוכן
    for c in df.columns:
        vals = " ".join(map(str, df[c].dropna().astype(str).head(15).tolist())).lower()
        if "wikipedia.org" in vals or "https://" in vals or "http://" in vals:
            return c
    return None

def http_get(url, params=None, headers=None, timeout=30, tries=3, backoff=0.7):
    headers = headers or UA
    for attempt in range(1, tries+1):
        try:
            r = session.get(url, params=params, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == tries:
                raise
            time.sleep(backoff * attempt + random.uniform(0, 0.2))

def guess_twitter_from_wiki(title_or_url):
    """
    נסה לחלץ ידית מהדף בוויקיפדיה:
    - External links
    - טקסט המקור (wikitext) עם regex
    """
    try:
        if isinstance(title_or_url, str) and title_or_url.startswith(("http://", "https://")):
            title = unquote(title_or_url.split("?")[0].split("#")[0].rstrip("/").split("/")[-1])
            title = title.replace("_", " ")
        else:
            title = str(title_or_url).strip().replace("_", " ")
        if not title:
            return None

        r = http_get(MEDIAWIKI_API, params={
            "action": "parse",
            "format": "json",
            "page": title,
            "prop": "externallinks|wikitext",
            "redirects": 1
        }, timeout=30, tries=3)
        data = r.json()

        links = data.get("parse", {}).get("externallinks", []) or []
        for ln in links:
            h = extract_handle_from_url(ln)
            if h:
                return h

        wt = data.get("parse", {}).get("wikitext", {}).get("*", "")
        # תופס גם twitter וגם x.com
        for m in re.finditer(r"(?:https?://)?(?:www\.)?(?:twitter|x)\.com/([A-Za-z0-9_]{1,15})", wt, re.I):
            h = clean_handle(m.group(1))
            if h:
                return h
    except:
        pass
    return None

def batch_p2002(qids):
    """
    מחזיר מפה { QID: handle or None } עם retry.
    """
    qids = [q for q in qids if isinstance(q, str) and q]
    if not qids:
        return {}
    values = " ".join(f"(wd:{q})" for q in qids)
    q = f"""
    SELECT ?item ?twitter WHERE {{
      VALUES (?item) {{ {values} }}
      OPTIONAL {{ ?item wdt:P2002 ?twitter. }}
    }}
    """
    # retry דרך http_get
    r = http_get(SPARQL, params={"query": q, "format": "json"}, timeout=45, tries=3, backoff=0.9)
    rows = r.json().get("results", {}).get("bindings", [])
    out = {}
    for b in rows:
        qid = b["item"]["value"].rsplit("/",1)[-1]
        tw = b.get("twitter", {}).get("value")
        out[qid] = clean_handle(tw) if tw else None
    return out

# cache (QID -> handle)
cache_file = os.path.join(ROOT, "_twitter_cache.json")
twitter_cache = {}
if os.path.exists(cache_file):
    try:
        twitter_cache = json.load(open(cache_file, "r", encoding="utf-8"))
    except:
        twitter_cache = {}

# ---------- process all subfolders ----------
folders = [os.path.join(ROOT, d) for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))]
folders.sort()

for FOLDER in folders:
    in_csv = pick_csv(FOLDER)
    if not in_csv:
        print(f"⚠️ אין CSV בתיקייה {os.path.basename(FOLDER)} — דילוג")
        continue

    print(f"\n📂 {os.path.basename(FOLDER)}")
    try:
        df = pd.read_csv(in_csv)
    except Exception as e:
        print(f"❌ לא הצלחתי לקרוא את הקובץ: {e} — דילוג")
        continue

    wiki_col = find_wikipedia_column(df)
    if "wikidata_qid" not in df.columns:
        df["wikidata_qid"] = None

    if "Twitter_username" not in df.columns:
        df["Twitter_username"] = None

    # 1) Wikidata P2002 (with cache)
    qids = [q for q in df["wikidata_qid"].dropna().astype(str).unique() if q]
    p2002_map = {}
    # קודם מה-cache
    for q in qids:
        if q in twitter_cache:
            p2002_map[q] = twitter_cache[q]
    # מה שחסר — לשאילתא (בקבוצות)
    to_query = [q for q in qids if q not in p2002_map]
    for i in range(0, len(to_query), 60):
        part = to_query[i:i+60]
        try:
            m = batch_p2002(part)
            p2002_map.update(m)
            # לעדכן cache רק כשיש ידית (כדי לא “לקבע” None)
            twitter_cache.update({k: v for k, v in m.items() if v})
            time.sleep(0.25)
        except Exception as e:
            print("⚠️ SPARQL batch failed:", e)

    # כתיבה לפי QID
    for idx, row in df.iterrows():
        qid = row.get("wikidata_qid")
        if isinstance(qid, str) and qid in p2002_map and p2002_map[qid]:
            if not df.at[idx, "Twitter_username"]:
                df.at[idx, "Twitter_username"] = p2002_map[qid]

    # 2) Wikipedia fallback
    if wiki_col:
        missing = df["Twitter_username"].isna()
        for idx, row in df[missing].iterrows():
            url = row[wiki_col]
            if isinstance(url, str) and url.strip():
                h = guess_twitter_from_wiki(url)
                if h:
                    df.at[idx, "Twitter_username"] = h
                    # אם יש גם QID — עדכן cache (נחסוך בהפעלות עתידיות)
                    qid = row.get("wikidata_qid")
                    if isinstance(qid, str) and qid:
                        twitter_cache[qid] = h

    # ניקוי + URL
    df["Twitter_username"] = df["Twitter_username"].apply(lambda h: clean_handle(h) if isinstance(h, str) else None)
    df["Twitter_url"] = df["Twitter_username"].apply(build_twitter_url)

    # save (UTF-8-SIG to be Excel-friendly)
    out_path = os.path.join(FOLDER, os.path.splitext(os.path.basename(in_csv))[0] + "_with_twitter.csv")
    try:
        df.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"✅ Saved: {os.path.basename(out_path)} | found {df['Twitter_username'].notna().sum()} / {len(df)}")
    except Exception as e:
        print(f"❌ שמירה נכשלה: {e}")

# save cache
try:
    with open(cache_file, "w", encoding="utf-8") as f:
        json.dump(twitter_cache, f, ensure_ascii=False, indent=2)
    print("\n💾 cache saved:", cache_file)
except Exception as e:
    print("⚠️ couldn't save cache:", e)

print("\n🎉 Done — Step 4 completed for all POI folders (username + url only).")


In [ ]:
# ============================================
# Build Manual_Search_POIs.csv from all *_with_twitter.csv
# ממלא אוטומטית את הקובץ המרכזי מכל התיקיות (מהעמודה Twitter_username)
# בהתאם להנחיות הקורס: שם הקובץ Manual_Search_POIs.csv ושמירה תחת POIs/
# ============================================

from google.colab import drive
import os, re, glob, pandas as pd

# ---- Mount + locate shared 'Iran' folder (dynamic, portable) ----
drive.mount('/content/drive', force_remount=False)

def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = os.environ.get('IRAN_DIR')
if not BASE or not os.path.isdir(BASE):
    BASE = find_shared_folder('Iran')

assert BASE and os.path.isdir(BASE), "❌ לא נמצאה תיקיית 'Iran' בדרייב."
REL_ROOT = "POIs"
ROOT = os.path.join(BASE, REL_ROOT)
assert os.path.isdir(ROOT), f"❌ לא קיימת התיקייה: {ROOT}"
print("📁 ROOT:", ROOT)

# שם הקובץ לפי ההנחיות (שלב 5)
OUTPUT_NAME = "Manual_Search_POIs.csv"
OUTPUT_PATH = os.path.join(ROOT, OUTPUT_NAME)

# -------- Keyword map (folder_slug -> keyword) --------
# הערה: השמות כאן ב-lowercase כי התיקיות נוצרו עם slug קטן (safe_slug)
KEYWORD_MAP = {
    # Existing
    "government_ministers_of_iran": "Iran minister",
    "presidents_of_iran": "Iran president",
    "vice_presidents_of_iran": "Iran vice president",
    "iranian_ayatollahs": "Iran ayatollah",
    "iranian_actors": "Iran actor",
    "iranian_singers": "Iran singer",
    "iranian_scientists": "Iran scientist",
    "iranian_economists": "Iran economist",
    "iranian_writers": "Iran writer",
    "iranian_football_managers": "Iran football manager",

    # Healthcare related
    "hospitals_in_iran": "Iran hospital",
    "private_hospitals_in_iran": "Iran hospital",
    "teaching_hospitals_in_iran": "Iran teaching hospital",
    "iranian_physicians": "Iran physician",
    "iranian_cardiologists": "Iran cardiologist",
    "iranian_women_physicians": "Iran physician",
    "21st-century_iranian_physicians": "Iran physician",
    "19th-century_iranian_physicians": "Iran physician",
    "medical_and_health_organisations_based_in_iran": "Iran health org",
    "healthcare_in_iran": "Iran healthcare",
    "medicine_in_iran": "Iran medicine",
}

CENTURY_PREFIX = re.compile(r"^\d{1,2}(st|nd|rd|th)-century_")

def base_slug(folder_slug: str) -> str:
    """מסיר קידומת של מאה מהסלאג (e.g., 19th-century_...)"""
    return CENTURY_PREFIX.sub("", folder_slug)

def infer_keyword_from_folder(folder_slug: str) -> str:
    """מפיק מילת חיפוש הגיונית מהסלאג, עם מפה ידנית לקייסים חשובים."""
    slug = folder_slug.lower().strip()
    if slug in KEYWORD_MAP:
        return KEYWORD_MAP[slug]
    slug = base_slug(slug)
    if slug in KEYWORD_MAP:
        return KEYWORD_MAP[slug]
    # כלל ברירת מחדל: הורד "iranian_" וה־s הסופית, והחלף _ ברווח
    token = slug.replace("iranian_", "").replace("_", " ").strip()
    token = token[:-1] if token.endswith("s") else token
    return f"Iran {token}".strip()

def detect_name_col(df: pd.DataFrame) -> str:
    """מאתר עמודת שם סבירה."""
    for c in df.columns:
        n = str(c).strip().lower()
        if n in {"name", "poi name", "poi_name", "full_name", "full name"}:
            return c
    # fallback: העמודה הראשונה
    return df.columns[0]

def detect_twitter_col(df: pd.DataFrame) -> str | None:
    """מאתר עמודת Twitter_username (או דומה)."""
    for c in df.columns:
        n = str(c).strip().lower()
        if n in {"twitter_username", "twitter", "handle", "username"}:
            return c
    # חפש עמודה שיש בה הרבה ערכים שנראים כמו ידיות
    for c in df.columns:
        vals = df[c].dropna().astype(str).head(20).tolist()
        hits = sum(1 for v in vals if re.match(r"^@?[A-Za-z0-9_]{1,15}$", v))
        if hits >= max(3, len(vals)//3):
            return c
    return None

def build_twitter_url(handle: str) -> str | None:
    if not isinstance(handle, str) or not handle.strip():
        return None
    h = handle.strip()
    h = h[1:] if h.startswith("@") else h
    return f"https://x.com/{h}"

rows = []
folders = [d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))]
folders.sort()

for folder in folders:
    folder_path = os.path.join(ROOT, folder)
    # חפש את קובץ היעד שנוצר בשלב 4
    cands = sorted(glob.glob(os.path.join(folder_path, "*_with_twitter.csv")))
    if not cands:
        continue
    path = cands[0]

    try:
        df = pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="utf-8-sig")
    except Exception as e:
        print("⚠️ can't read:", path, e)
        continue

    twitter_col = detect_twitter_col(df)
    if not twitter_col:
        continue

    name_col = detect_name_col(df)
    kw = infer_keyword_from_folder(folder)

    sub = df[[name_col, twitter_col]].copy()
    sub.rename(columns={name_col: "poi_name", twitter_col: "Twitter_username"}, inplace=True)

    # ניקוי, סינון, והשלמת URL
    sub["Twitter_username"] = sub["Twitter_username"].astype(str).str.strip()
    sub = sub[sub["Twitter_username"].str.len() > 0]
    sub["Twitter_username"] = sub["Twitter_username"].str.lstrip("@")
    sub["Twitter_url"] = sub["Twitter_username"].apply(build_twitter_url)
    sub["keyword"] = kw
    sub["source_folder"] = folder  # מאיזה קטגוריה הגיע

    rows.append(sub[["keyword", "poi_name", "Twitter_username", "Twitter_url", "source_folder"]])

# איחוד ושמירה
if rows:
    out = pd.concat(rows, ignore_index=True)
    # סינון כפילויות על בסיס שלישייה (שם משתמש + מילת חיפוש + שם POI)
    out.drop_duplicates(subset=["Twitter_username", "keyword", "poi_name"], inplace=True, ignore_index=True)
    # וריאנט נוסף: אם אותו handle הופיע בכמה תיקיות — נשמור את ההופעה הראשונה בלבד
    out.sort_values(["Twitter_username", "keyword", "poi_name"], inplace=True, ignore_index=True)
    out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
    print(f"✅ Saved: {OUTPUT_PATH} | rows: {len(out)}")
    try:
        from IPython.display import display
        display(out.head(20))
    except Exception:
        pass
else:
    print("ℹ️ לא נמצאו *_with_twitter.csv בתיקיות, או שאין ידיות למלא.")


#**Cleaning up suspended users**

In [ ]:
# # ============================================
# # Update POIs_Search_Manual.csv:
# # - Find the correct Twitter handle column (robust name detection)
# # - Replace numeric-only handles with "Suspended"
# # - Rebuild Twitter_url accordingly
# # ============================================

# import os, re, pandas as pd

# BASES = [
#     "/content/drive/MyDrive/Iran",
#     "/content/drive/MyDrive/data_science_semester_A/Iran",
# ]
# REL_ROOT   = "POIs"
# OUTPUT_NAME = "POIs_Search_Manual.csv"

# # 1) Locate file
# BASE = next((b for b in BASES if os.path.exists(b)), None)
# assert BASE, "Drive base not found. Mount Google Drive first."
# ROOT = os.path.join(BASE, REL_ROOT)
# assert os.path.isdir(ROOT), f"Folder not found: {ROOT}"
# path = os.path.join(ROOT, OUTPUT_NAME)
# assert os.path.exists(path), f"File not found: {path}"
# print("📄 Using file:", path)

# # 2) Load
# df = pd.read_csv(path)

# # 3) Detect the twitter username column robustly (case-insensitive, common variants)
# cands = [c for c in df.columns if re.sub(r'\s+', '', c.strip().lower()) in {
#     "twitter_username","twitteruser","twitter_user","twitterhandle","handle","username"
# }]
# if not cands:
#     # fallback: try any column containing 'twitter' and 'name'
#     cands = [c for c in df.columns if "twitter" in c.lower() and "name" in c.lower()]
# assert cands, f"Could not find Twitter username column in {list(df.columns)}"
# tw_col = cands[0]
# print("✅ Detected username column:", tw_col)

# # 4) Replace numeric-only usernames with "Suspended"
# before = df[tw_col].astype(str)
# df[tw_col] = df[tw_col].apply(lambda v: "Suspended" if isinstance(v, str) and re.fullmatch(r"\d+", v.strip() or "") else v)
# changed_usernames = (before != df[tw_col].astype(str)).sum()

# # 5) Rebuild Twitter_url consistently (create if missing)
# url_col = "Twitter_url" if "Twitter_url" in df.columns else None
# if url_col is None:
#     df["Twitter_url"] = None
#     url_col = "Twitter_url"

# def url_from_handle(h):
#     if not isinstance(h, str): return None
#     h = h.strip().lstrip("@")
#     if h == "" or h.lower() == "suspended": return None
#     return f"https://x.com/{h}"

# before_urls = df[url_col].copy()
# df[url_col] = df[tw_col].apply(url_from_handle)
# fixed_urls = (before_urls != df[url_col]).sum()

# # 6) Save
# df.to_csv(path, index=False, encoding="utf-8-sig")
# print(f"✅ Done. Replaced {changed_usernames} usernames with 'Suspended', updated {fixed_urls} URLs.")

# # Peek
# try:
#     from IPython.display import display
#     display(df.head(15))
# except:
#     print(df.head(15))


In [ ]:
# ============================================
# Build Manual_Search_POIs.csv from all *_with_twitter.csv
# ממלא אוטומטית את הקובץ המרכזי מכל התיקיות (Twitter_username)
# ============================================

from google.colab import drive
from IPython.display import display
import os, glob, re, pandas as pd

# ---- Mount Drive ----
drive.mount('/content/drive', force_remount=False)

# ---- Locate (or create if missing) shared 'Iran' folder ----
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
if not BASE:
    BASE = '/content/drive/MyDrive/Iran'
    os.makedirs(BASE, exist_ok=True)
    print("⚠️ לא נמצאה תיקיית 'Iran' משותפת — נוצרה חדשה מקומית:", BASE)
else:
    print("📁 נמצאה תיקיית Iran:", BASE)

REL_ROOT    = "POIs"
ROOT        = os.path.join(BASE, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
OUTPUT_NAME = "Manual_Search_POIs.csv"   # שם לפי ההנחיות
OUTPUT_PATH = os.path.join(ROOT, OUTPUT_NAME)

# ---- KEYWORD MAP: רק הקטגוריות שביקשת (לפי slug בתיקיות - lowercase) ----
KEYWORD_MAP = {
    # Existing
    "government_ministers_of_iran": "Iran minister",
    "presidents_of_iran": "Iran president",
    "vice_presidents_of_iran": "Iran vice president",
    "iranian_ayatollahs": "Iran ayatollah",
    "iranian_actors": "Iran actor",
    "iranian_singers": "Iran singer",
    "iranian_scientists": "Iran scientist",
    "iranian_economists": "Iran economist",
    "iranian_writers": "Iran writer",
    "iranian_football_managers": "Iran football manager",

    # Healthcare
    "hospitals_in_iran": "Iran hospital",
    "private_hospitals_in_iran": "Iran hospital",
    "teaching_hospitals_in_iran": "Iran teaching hospital",
    "iranian_physicians": "Iran physician",
    "iranian_cardiologists": "Iran cardiologist",
    "iranian_women_physicians": "Iran physician",
    "21st-century_iranian_physicians": "Iran physician",
    "19th-century_iranian_physicians": "Iran physician",
    "medical_and_health_organisations_based_in_iran": "Iran health org",
    "healthcare_in_iran": "Iran healthcare",
    "medicine_in_iran": "Iran medicine",
}

CENTURY_PREFIX = re.compile(r"^\d{1,2}(st|nd|rd|th)-century_")

def normalize_folder_slug(folder_name: str) -> str:
    """מוריד רווחים/רישיות, מסיר קידומת מאה."""
    slug = folder_name.strip().lower()
    slug = CENTURY_PREFIX.sub(lambda m: m.group(0), slug)  # משאיר את קידומת המאה אם קיימת (כדי להתאים ל-map)
    return slug

def base_slug(folder_slug: str) -> str:
    """מסיר קידומת מאה לשימוש ברירת מחדל."""
    return CENTURY_PREFIX.sub("", folder_slug)

def infer_keyword_from_folder(folder: str) -> str:
    slug = normalize_folder_slug(folder)
    # קודם מיפוי מדויק
    if slug in KEYWORD_MAP:
        return KEYWORD_MAP[slug]
    # נסה בלי קידומת מאה
    bslug = base_slug(slug)
    if bslug in KEYWORD_MAP:
        return KEYWORD_MAP[bslug]
    # ברירת מחדל: הפוך ל־"Iran <token>"
    token = bslug.replace("iranian_", "").replace("_", " ").strip()
    token = token[:-1] if token.endswith("s") else token
    return f"Iran {token}".strip()

def detect_name_col(df: pd.DataFrame) -> str:
    for c in df.columns:
        n = str(c).strip().lower()
        if n in {"name", "poi name", "poi_name", "full_name", "full name"}:
            return c
    return df.columns[0]

# ---- Aggregate from all *_with_twitter.csv under ROOT/* ----
rows = []
folders = sorted([d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))])
print(f"📂 נמצאו {len(folders)} תיקיות מתחת ל-{REL_ROOT}")

for folder in folders:
    folder_path = os.path.join(ROOT, folder)
    cands = sorted(glob.glob(os.path.join(folder_path, "*_with_twitter.csv")))
    if not cands:
        continue

    csv_path = cands[0]
    try:
        df = pd.read_csv(csv_path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(csv_path, encoding="utf-8-sig")
    except Exception as e:
        print("⚠️ can't read:", csv_path, e)
        continue

    if "Twitter_username" not in df.columns:
        continue

    name_col = detect_name_col(df)
    kw = infer_keyword_from_folder(folder)

    sub = df[[name_col, "Twitter_username"]].dropna(subset=["Twitter_username"]).copy()
    sub.rename(columns={name_col: "poi_name"}, inplace=True)
    sub["Twitter_username"] = sub["Twitter_username"].astype(str).str.strip().str.lstrip("@")
    sub = sub[sub["Twitter_username"].str.len() > 0]
    sub["Twitter_url"] = "https://x.com/" + sub["Twitter_username"]
    sub["keyword"] = kw
    sub["source_folder"] = folder

    rows.append(sub[["keyword", "poi_name", "Twitter_username", "Twitter_url", "source_folder"]])

# ---- Save merged CSV ----
if rows:
    out = pd.concat(rows, ignore_index=True)
    out.drop_duplicates(subset=["Twitter_username", "keyword", "poi_name"], inplace=True, ignore_index=True)
    out.sort_values(["keyword", "poi_name"], inplace=True, ignore_index=True)
    out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
    print(f"✅ Saved: {OUTPUT_PATH} | rows: {len(out)}")
    display(out.head(20))
else:
    print("ℹ️ לא נמצאו *_with_twitter.csv בתיקיות, או שאין ידיות למלא.")


הוא צעד עזר טוב לפני שלב 6.


In [ ]:
# ============================================
# Add "Twitter_status" column to mark suspended/invalid/missing users
# Works on Manual_Search_POIs.csv
# ============================================

from google.colab import drive
import os, re, pandas as pd
from IPython.display import display

# ---- Mount Google Drive ----
drive.mount('/content/drive', force_remount=False)

# ---- Locate (or create if missing) shared 'Iran' folder ----
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
if not BASE:
    BASE = '/content/drive/MyDrive/Iran'
    os.makedirs(BASE, exist_ok=True)
    print("⚠️ לא נמצאה תיקיית 'Iran' משותפת — נוצרה חדשה מקומית:", BASE)
else:
    print("📁 נמצאה תיקיית Iran:", BASE)

# ---- Config ----
REL_ROOT  = "POIs"
FILE_NAME = "Manual_Search_POIs.csv"   # <-- מעודכן לשם הקובץ החדש

ROOT = os.path.join(BASE, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
path = os.path.join(ROOT, FILE_NAME)
assert os.path.exists(path), f"❌ File not found: {path}"

# ---- Load ----
try:
    df = pd.read_csv(path, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding="utf-8-sig")

# ---- Helpers ----
HANDLE_RE = re.compile(r"^[A-Za-z0-9_]{1,15}$")

def classify_status(username: str) -> str:
    if pd.isna(username):
        return "missing"
    s = str(username).strip().lstrip("@")
    if not s:
        return "missing"
    # מספר בלבד: לעיתים מצביע על חשבון מושעה/מומר למזהה
    if s.isdigit():
        return "suspended"
    # ידית לא חוקית (אורך >15 או תווים לא תקינים/רווחים)
    if (len(s) > 15) or (HANDLE_RE.fullmatch(s) is None):
        return "invalid"
    return "active"

# ---- Compute & Save ----
df["Twitter_status"] = df["Twitter_username"].apply(classify_status)

df.to_csv(path, index=False, encoding="utf-8-sig")
print(f"✅ Updated file: {path} ({len(df)} rows)")
try:
    display(df.head(20))
except Exception:
    pass


In [ ]:
# ============================================
# 🧮 Step 6 – Enhanced Statistics & Visuals (UPDATED)
# Builds robust stats table + coverage + charts (portable)
# ============================================

from google.colab import drive
from IPython.display import display
import os, glob, re, pandas as pd
import matplotlib.pyplot as plt

# ---------- Mount & locate base (dynamic) ----------
drive.mount('/content/drive', force_remount=False)

def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
if not BASE:
    BASE = '/content/drive/MyDrive/Iran'           # יצירה להרצה ראשונה אם אין משותפת
    os.makedirs(BASE, exist_ok=True)
    print("⚠️ לא נמצאה תיקיית 'Iran' משותפת — נוצרה מקומית:", BASE)
else:
    print("📁 נמצאה תיקיית Iran:", BASE)

REL_ROOT     = "POIs"
OUT_BASENAME = "POI_statistics.csv"             # basic
OUT_ENHANCED = "POI_statistics_enhanced.csv"    # enhanced with coverage
FIG_DIR      = "figures"

ROOT = os.path.join(BASE, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)

# ---------- Helpers ----------
def safe_read_csv(path, nrows=None):
    for enc in (None, "utf-8", "utf-8-sig"):
        try:
            return pd.read_csv(path, nrows=nrows, encoding=enc)
        except Exception:
            pass
    # ניסיון אחרון עם engine=python ו-skip
    try:
        return pd.read_csv(path, nrows=nrows, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
    except Exception:
        return None

def is_wikipedia_csv(path):
    """Detect Wikipedia CSV by content (columns/values), not filename alone."""
    name = os.path.basename(path).lower()
    if any(k in name for k in ["wikidata", "with_twitter"]):
        return False
    df = safe_read_csv(path, nrows=10)
    if df is None or df.empty:
        return False
    cols = [str(c).strip().lower() for c in df.columns]
    if any(any(k in c for k in ["wikipedia", "wiki", "link", "url"]) for c in cols):
        return True
    for c in df.columns:
        try:
            sample = " ".join(map(str, df[c].dropna().astype(str).head(10).tolist())).lower()
            if "wikipedia.org" in sample or sample.startswith("http"):
                return True
        except Exception:
            pass
    return False

def detect_twitter_col(df):
    candidates = {"twitter_username","twitter user","twitter_user","twitter handle","handle","twitter"}
    for c in df.columns:
        if str(c).strip().lower() in candidates:
            return c
    for c in df.columns:
        norm = re.sub(r'[^a-z]', '', str(c).lower())
        if "twitter" in norm and ("username" in norm or "handle" in norm or "user" in norm or "name" in norm):
            return c
    return None

def detect_qid_col(df):
    for c in df.columns:
        if str(c).strip().lower() in {"wikidata_qid","qid","wikidata id","wikidataid"}:
            return c
    return None

# ---------- Build basic stats ----------
rows = []
folders = sorted([d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))])

for folder in folders:
    fpath = os.path.join(ROOT, folder)
    csvs = sorted(glob.glob(os.path.join(fpath, "*.csv")))
    if not csvs:
        continue

    wiki_file = None
    wikidata_file = None
    twitter_file = None

    # Twitter file by name
    for p in csvs:
        if re.search(r"with_twitter", os.path.basename(p), re.I):
            twitter_file = p
            break

    # Wikidata: prefer detailed
    for p in csvs:
        if re.search(r"wikidata.*detailed", os.path.basename(p), re.I):
            wikidata_file = p
            break
    if wikidata_file is None:
        for p in csvs:
            if re.search(r"with_wikidata_ids_and_links", os.path.basename(p), re.I):
                wikidata_file = p
                break

    # Wikipedia by content (fallback to "<folder>.csv")
    for p in csvs:
        if is_wikipedia_csv(p):
            wiki_file = p
            break
    if wiki_file is None:
        fallback = os.path.join(fpath, f"{folder}.csv")
        if os.path.exists(fallback):
            wiki_file = fallback

    wikipedia_count = 0
    wikidata_count  = 0
    twitter_count   = 0

    # Wikipedia count
    if wiki_file:
        dfw = safe_read_csv(wiki_file)
        if dfw is not None:
            wikipedia_count = len(dfw)

    # Wikidata count (count non-empty QIDs if column exists)
    if wikidata_file:
        dfd = safe_read_csv(wikidata_file)
        if dfd is not None:
            qcol = detect_qid_col(dfd)
            wikidata_count = dfd[qcol].notna().sum() if qcol else len(dfd)

    # Twitter count (valid handles only)
    if twitter_file:
        dft = safe_read_csv(twitter_file)
        if dft is not None:
            tw_col = detect_twitter_col(dft)
            if tw_col:
                s = dft[tw_col].astype(str).str.strip()
                valid = s[(s != "") &
                          (~s.str.fullmatch(r"\d+")) &
                          (~s.str.lower().isin({"nan","none","null","suspended"}))]
                twitter_count = valid.shape[0]

    rows.append({
        "Category": folder,
        "Wikipedia count": wikipedia_count,
        "Wikidata count": wikidata_count,
        "Twitter user count": twitter_count
    })

basic = pd.DataFrame(rows).sort_values("Category").reset_index(drop=True)
basic_path = os.path.join(ROOT, OUT_BASENAME)
basic.to_csv(basic_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved basic table: {basic_path}")
display(basic)

# ---------- Enhanced: coverage, gaps, totals ----------
enh = basic.copy()
with pd.option_context('mode.use_inf_as_na', True):
    enh["Wikidata coverage (%)"] = (100 * enh["Wikidata count"] / enh["Wikipedia count"]).fillna(0).round(1)
    enh["Twitter coverage (%)"]  = (100 * enh["Twitter user count"] / enh["Wikipedia count"]).fillna(0).round(1)

enh["Wikidata gap"] = (enh["Wikipedia count"] - enh["Wikidata count"]).clip(lower=0)
enh["Twitter gap"]  = (enh["Wikipedia count"] - enh["Twitter user count"]).clip(lower=0)

tot = pd.DataFrame([{
    "Category": "TOTAL",
    "Wikipedia count": enh["Wikipedia count"].sum(),
    "Wikidata count": enh["Wikidata count"].sum(),
    "Twitter user count": enh["Twitter user count"].sum(),
}])
tot["Wikidata coverage (%)"] = (100 * tot["Wikidata count"] / tot["Wikipedia count"]).round(1) if tot["Wikipedia count"].iloc[0] else 0.0
tot["Twitter coverage (%)"]  = (100 * tot["Twitter user count"] / tot["Wikipedia count"]).round(1) if tot["Wikipedia count"].iloc[0] else 0.0
tot["Wikidata gap"] = tot["Wikipedia count"] - tot["Wikidata count"]
tot["Twitter gap"]  = tot["Wikipedia count"] - tot["Twitter user count"]

enhanced = pd.concat([enh, tot], ignore_index=True)
enhanced_path = os.path.join(ROOT, OUT_ENHANCED)
enhanced.to_csv(enhanced_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved enhanced table: {enhanced_path}")
display(enhanced)

# ---------- Charts ----------
fig_dir = os.path.join(ROOT, FIG_DIR)
os.makedirs(fig_dir, exist_ok=True)

# מסננים את TOTAL עבור גרפים פר-קטגוריה
per_cat = enhanced[enhanced["Category"] != "TOTAL"].reset_index(drop=True)

# 1) Counts by category (lines) — ללא TOTAL
plt.figure(figsize=(12,5))
for col in ["Wikipedia count", "Wikidata count", "Twitter user count"]:
    plt.plot(per_cat["Category"], per_cat[col], marker='o', label=col)
plt.xticks(rotation=45, ha='right')
plt.title("POIs – Counts by Source")
plt.xlabel("Category")
plt.ylabel("Count")
plt.legend()
plt.tight_layout()
fig1_path = os.path.join(fig_dir, "counts_by_category.png")
plt.savefig(fig1_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig1_path}")

# 2) Coverage by category (%) — ללא TOTAL
plt.figure(figsize=(12,5))
plt.bar(per_cat["Category"], per_cat["Wikidata coverage (%)"], alpha=0.8, label="Wikidata coverage (%)")
plt.bar(per_cat["Category"], per_cat["Twitter coverage (%)"], alpha=0.6, label="Twitter coverage (%)")
plt.xticks(rotation=45, ha='right')
plt.title("POIs – Coverage by Category (%)")
plt.xlabel("Category")
plt.ylabel("Coverage (%)")
plt.legend()
plt.tight_layout()
fig2_path = os.path.join(fig_dir, "coverage_by_category.png")
plt.savefig(fig2_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig2_path}")

# 3) Stacked bars: gaps to Wikipedia — ללא TOTAL
plt.figure(figsize=(12,5))
bar_x = range(len(per_cat))
plt.bar(bar_x, per_cat["Wikidata count"], label="Wikidata count")
plt.bar(bar_x, per_cat["Wikidata gap"], bottom=per_cat["Wikidata count"], label="Wikidata gap")
plt.xticks(bar_x, per_cat["Category"], rotation=45, ha='right')
plt.title("Wikidata Count + Gap to Wikipedia")
plt.xlabel("Category")
plt.ylabel("Entities")
plt.legend()
plt.tight_layout()
fig3_path = os.path.join(fig_dir, "wikidata_gap_stacked.png")
plt.savefig(fig3_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig3_path}")

print("\n✅ Done. Files written to:")
print(f"- {basic_path}")
print(f"- {enhanced_path}")
print(f"- {fig1_path}")
print(f"- {fig2_path}")
print(f"- {fig3_path}")


In [ ]:
!pip uninstall -y snscrape
!pip install git+https://github.com/JustAnotherArchivist/snscrape.git@master


# *Build POI_twitter_users_data.csv from POIs_Search_Manual*.csv

In [ ]:
# ============================================================
# STEP 7 (UPDATED) — Build POI_twitter_users_data.csv
# Source: Manual_Search_POIs.csv  (not POIs_Search_Manual*)
# Portable: dynamic Drive path detection, robust CSV reading
# ============================================================

# 0) Mount Drive (safe if already mounted)
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# 0.1) Locate (or create if missing) shared 'Iran' folder + POIs root
import os, re, glob
from pathlib import Path

def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
if not BASE:
    BASE = '/content/drive/MyDrive/Iran'
    os.makedirs(BASE, exist_ok=True)
    print("⚠️ לא נמצאה תיקיית 'Iran' משותפת — נוצרה חדשה מקומית:", BASE)
else:
    print("📁 נמצאה תיקיית Iran:", BASE)

REL = "POIs"
POIs_DIR = os.path.join(BASE, REL)
os.makedirs(POIs_DIR, exist_ok=True)

# 1) Quiet logs + progress bar
import logging, warnings
logging.getLogger("snscrape").setLevel(logging.ERROR)
logging.getLogger("snscrape.modules.twitter").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm.auto import tqdm
from IPython.display import display

# 2) SSL & deps
import sys, subprocess, importlib, site, pathlib, time, random
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "certifi", "requests", "urllib3", "idna", "tqdm"], check=True)
import certifi
os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()

# 3) Install/patch snscrape for Py3.12 (if needed)
def ensure_snscrape():
    try:
        import snscrape.modules.twitter as sntwitter
        return sntwitter
    except Exception:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/JustAnotherArchivist/snscrape.git@master"], check=True)
        # patch legacy importer if needed
        import pkgutil
        for sp in set(site.getsitepackages() + [site.getusersitepackages()]):
            p = pathlib.Path(sp) / "snscrape" / "modules" / "__init__.py"
            if p.exists():
                txt = p.read_text(encoding="utf-8")
                if "find_module(" in txt or "load_module(" in txt:
                    import re as _re
                    patched = _re.sub(
                        r'(\s*)module\s*=\s*importer\.find_module\(moduleName\)\.load_module\(moduleName\)',
                        r'\1import importlib\n\1module = importlib.import_module(moduleName)',
                        txt
                    )
                    if "prefixLen" not in patched:
                        patched = _re.sub(
                            r'for importer, moduleName, ispkg in pkgutil\.iter_modules\(__path__, prefix\):',
                            'prefixLen = len(prefix)\n    for importer, moduleName, ispkg in pkgutil.iter_modules(__path__, prefix):',
                            patched
                        )
                    p.write_text(patched, encoding="utf-8")
        importlib.invalidate_caches()
        import snscrape.modules.twitter as sntwitter
        return sntwitter

sntwitter = ensure_snscrape()

# 4) Locate Manual_Search_POIs.csv under POIs_DIR
import pandas as pd

def safe_read_csv(path):
    for enc in (None, "utf-8", "utf-8-sig"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    try:
        return pd.read_csv(path, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
    except Exception as e:
        raise

def find_manual_csv(root_dir: str) -> str:
    # חפש בדיוק Manual_Search_POIs.csv קודם
    exact = os.path.join(root_dir, "Manual_Search_POIs.csv")
    if os.path.exists(exact):
        return exact
    # fallback: כל קובץ בשם Manual_Search_POIs*.csv
    candidates = []
    for dp, _, files in os.walk(root_dir):
        for fn in files:
            if re.match(r"Manual_Search_POIs.*\.csv$", fn, flags=re.I):
                full = os.path.join(dp, fn)
                try:
                    mtime = os.path.getmtime(full)
                except Exception:
                    mtime = 0
                candidates.append((mtime, full))
    if not candidates:
        raise FileNotFoundError(f"לא נמצא קובץ בשם 'Manual_Search_POIs*.csv' תחת: {root_dir}")
    candidates.sort(reverse=True)
    return candidates[0][1]

MANUAL_CSV = find_manual_csv(POIs_DIR)
print("📄 Manual CSV:", MANUAL_CSV)

# 5) Output side-by-side to POIs root
OUT_CSV = os.path.join(POIs_DIR, "POI_twitter_users_data.csv")
print("📁 Output CSV :", OUT_CSV)

# 6) Load manual CSV & detect twitter/name/status columns
df = safe_read_csv(MANUAL_CSV)

def looks_like_twitter_col(name: str) -> bool:
    n = name.lower().strip()
    keys = ["twitter", "x", "handle", "username", "tw", "טוויטר"]
    return any(k == n or k in n for k in keys)

twitter_col = next((c for c in df.columns if looks_like_twitter_col(str(c))), None)
if twitter_col is None:
    for c in df.columns:
        sample = " ".join(map(str, df[c].dropna().astype(str).head(40).tolist())).lower()
        if "twitter.com" in sample or "x.com/" in sample or "@" in sample:
            twitter_col = c
            break
assert twitter_col, "לא מצאתי עמודת טוויטר."

status_col = next((c for c in df.columns if str(c).lower().strip() in ["twitter_status", "status", "tw_status"]), None)
name_col   = next((c for c in df.columns if str(c).lower().strip() in ["poi_name", "name", "full name", "fullname"]), None)

# 7) Extract usernames (max 15 chars per X rules)
import re as _re
HANDLE_RE = _re.compile(r"^[A-Za-z0-9_]{1,15}$")

def extract_username(val) -> str | None:
    if pd.isna(val): return None
    s = str(val).strip()
    if not s: return None
    s = s.replace("https://", "").replace("http://", "").replace("www.", "")
    m = _re.search(r"(?:twitter\.com|x\.com)/([A-Za-z0-9_]{1,15})", s, flags=_re.I)
    if m: return m.group(1)
    m = _re.search(r"@([A-Za-z0-9_]{1,15})", s)
    if m: return m.group(1)
    if HANDLE_RE.fullmatch(s): return s
    return None

rows = []
for _, row in df.iterrows():
    u = extract_username(row.get(twitter_col))
    if not u:
        continue
    rows.append({
        "username": u,
        "poi_name": row.get(name_col),
        "twitter_status": (str(row.get(status_col)).lower().strip() if status_col else None)
    })

assert rows, "לא נמצאו שמות משתמש תקינים."

# 8) Helpers: scraping / stub rows
from datetime import datetime, timezone

def g(obj, name, default=None):
    try: return getattr(obj, name, default)
    except Exception: return default

def safe_attr(obj, path, default=None):
    cur = obj
    try:
        for part in path.split("."):
            cur = getattr(cur, part, None)
            if cur is None: return default
        return cur
    except Exception:
        return default

def build_stub(u, poi, status, note):
    return {
        "full name": None,
        "description / bio": None,
        "followers_count": None,
        "following_count": None,
        "statuses_count": None,
        "created_at": None,
        "profile_image_url": None,
        "banner_url": None,
        "username": u,
        "external_url": None,
        "location": None,
        "verified": None,
        "protected": None,
        "language": None,
        "scraped_at": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "poi_name": poi,
        "twitter_status": status,
        "source": "csv_status_only",
        "error": note,
    }

def fetch_user_metadata(u: str):
    user_obj, last_err = None, None
    for ctor in [
        lambda x: sntwitter.TwitterUserScraper(x),           # positional
        lambda x: sntwitter.TwitterUserScraper(user=x),      # keyword (some builds)
        lambda x: sntwitter.XUserScraper(x) if hasattr(sntwitter, "XUserScraper") else (_ for _ in ()).throw(AttributeError("XUserScraper missing")),
    ]:
        try:
            user_obj = ctor(u).entity
            break
        except Exception as e:
            last_err = e
    if user_obj is None:
        raise RuntimeError(repr(last_err))

    return {
        "full name": g(user_obj, "displayname"),
        "description / bio": g(user_obj, "renderedDescription"),
        "followers_count": g(user_obj, "followersCount"),
        "following_count": g(user_obj, "friendsCount"),
        "statuses_count": g(user_obj, "statusesCount"),
        "created_at": g(user_obj, "created"),
        "profile_image_url": g(user_obj, "profileImageUrl"),
        "banner_url": g(user_obj, "bannerImageUrl"),
        "username": g(user_obj, "username"),
        "external_url": safe_attr(user_obj, "link.url"),
        "location": g(user_obj, "location"),
        "verified": g(user_obj, "verified"),
        "protected": g(user_obj, "protected"),
        "language": g(user_obj, "lang"),
        "scraped_at": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "poi_name": None,  # נמלא בהמשך
        "twitter_status": None,  # נמלא בהמשך
        "source": "snscrape",
        "error": None,
    }

# 9) Run with progress, retries, periodic saves
MAX_USERS  = None     # לדוגמה 100 לבדיקה; None = הכל
RETRIES    = 3
BASE_SLEEP = 1.0
SAVE_EVERY = 25       # שמירת ביניים כל N משתמשים

to_process = rows[:MAX_USERS] if MAX_USERS else rows
records = []
ok = suspended_cnt = failed = 0

cols_order = [
    "full name", "description / bio", "followers_count", "following_count",
    "statuses_count", "created_at", "profile_image_url", "banner_url",
    "username", "external_url", "location", "verified", "protected",
    "language", "scraped_at", "poi_name", "twitter_status", "source", "error"
]

pbar = tqdm(to_process, desc="Scraping users", unit="user")
for idx, r in enumerate(pbar, start=1):
    u = r["username"]; poi = r["poi_name"]; st = r["twitter_status"]

    # Respect suspended/invalid/missing from Step 5/6
    if st and any(k in st for k in ("suspend", "invalid", "missing")):
        records.append(build_stub(u, poi, st, note=f"status_from_manual_csv:{st}"))
        suspended_cnt += 1 if "suspend" in (st or "") else suspended_cnt
        pbar.set_postfix(ok=ok, suspended=suspended_cnt, failed=failed)
        continue

    for a in range(RETRIES):
        try:
            rec = fetch_user_metadata(u)
            rec["poi_name"] = poi
            rec["twitter_status"] = st
            records.append(rec)
            ok += 1
            break
        except Exception as e:
            if a == RETRIES - 1:
                records.append(build_stub(u, poi, st, note=f"scrape_failed: {e}"))
                failed += 1
            time.sleep(BASE_SLEEP + 0.6 * a + random.random() * 0.4)

    pbar.set_postfix(ok=ok, suspended=suspended_cnt, failed=failed)

    # periodic save
    if idx % SAVE_EVERY == 0:
        tmp_df = pd.DataFrame.from_records(records)
        tmp_df = tmp_df[[c for c in cols_order if c in tmp_df.columns]]
        if os.path.exists(OUT_CSV):
            old = safe_read_csv(OUT_CSV)
            merged = (pd.concat([old, tmp_df], ignore_index=True)
                      .drop_duplicates(subset=["username"], keep="last"))
        else:
            merged = tmp_df
        merged.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# 10) Final save/merge
out_df = pd.DataFrame.from_records(records)
out_df = out_df[[c for c in cols_order if c in out_df.columns]]

if os.path.exists(OUT_CSV):
    old = safe_read_csv(OUT_CSV)
    final = (pd.concat([old, out_df], ignore_index=True)
             .drop_duplicates(subset=["username"], keep="last"))
else:
    final = out_df

final.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"\n✅ DONE. Saved -> {OUT_CSV}")
print(f"   Totals: scraped={ok}, suspended/invalid/missing(stub)={suspended_cnt}, failed(stub)={failed}, total_written={len(final)}")

# Preview
try:
    display(final.tail(5))
except Exception:
    pass


# fill LANGUAGE with full names

In [ ]:
# ========= Post-process: fill LANGUAGE with full names (no abbreviations) =========
# ממלא שפת משתמשים משדה ה-bio היכן שחסר, וממיר קודים (en/fa/…) לשמות מלאים

# התקנות רק אם צריך
try:
    from langdetect import detect, DetectorFactory
except Exception:
    !pip -q install langdetect
    from langdetect import detect, DetectorFactory

try:
    import pycountry
except Exception:
    !pip -q install pycountry
    import pycountry

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, re, glob
import pandas as pd
from pathlib import Path
from IPython.display import display

DetectorFactory.seed = 0  # יציבות בזיהוי

# ---- איתור דינמי של תיקיית 'Iran' ושל קובץ ה-CSV תחת POIs ----
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran') or '/content/drive/MyDrive/Iran'
os.makedirs(BASE, exist_ok=True)
POIs_DIR = os.path.join(BASE, "POIs")
os.makedirs(POIs_DIR, exist_ok=True)

# הקובץ המרכזי שנוצר בשלב 7
CSV = Path(os.path.join(POIs_DIR, "POI_twitter_users_data.csv"))
if not CSV.exists():
    raise FileNotFoundError(f"❌ לא נמצא הקובץ: {CSV}")

# קריאה עמידה
try:
    df = pd.read_csv(CSV, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(CSV, encoding="utf-8-sig")

# ודא שקיימת עמודת language כמחרוזת
if "language" not in df.columns:
    df["language"] = ""
df["language"] = df["language"].astype("string")

# ---- זיהוי שפה מה-bio רק היכן שחסר/ריק ----
BIO_COL = "description / bio" if "description / bio" in df.columns else None

def detect_lang_from_bio(text):
    if pd.isna(text):
        return None
    s = str(text).strip()
    if len(s) < 6:  # קצר מדי, לא נזהה כדי להפחית false positives
        return None
    try:
        return detect(s)  # מחזיר קוד כמו 'en','fa','ar',...
    except Exception:
        return None

if BIO_COL:
    mask_missing = df["language"].isna() | (df["language"].str.strip().fillna("") == "")
    df.loc[mask_missing, "language"] = df.loc[mask_missing, BIO_COL].apply(detect_lang_from_bio)

# ---- המרה לשמות מלאים (לא מקצר שמות מלאים קיימים) ----
# תומך alpha-2/alpha-3 + נפוצים
MANUAL_MAP = {
    "en": "English",
    "fa": "Persian (Farsi)",
    "ar": "Arabic",
    "fr": "French",
    "es": "Spanish",
    "de": "German",
    "it": "Italian",
    "ru": "Russian",
    "tr": "Turkish",
    "ur": "Urdu",
    "pt": "Portuguese",
    "ja": "Japanese",
    "bg": "Bulgarian",
    "ro": "Romanian",
    "et": "Estonian",
    "sq": "Albanian",
    "so": "Somali",
    "id": "Indonesian",
    "da": "Danish",
    "sl": "Slovenian",
    "ca": "Catalan",
    "af": "Afrikaans",
    # alpha-3 נפוצים
    "fas": "Persian (Farsi)",
    "ara": "Arabic",
    "eng": "English",
    "spa": "Spanish",
    "deu": "German",
    "ger": "German",   # old biblio code
    "fra": "French",
    "fre": "French",   # old biblio code
    "rus": "Russian",
    "tur": "Turkish",
    "urd": "Urdu",
}

def is_code_like(v: str) -> bool:
    if not isinstance(v, str):
        return False
    s = v.strip()
    # קוד נראה כמו 2–3 אותיות לטיניות; לא כוללים שמות מלאים/מילים עם רווח
    return bool(re.fullmatch(r"[A-Za-z]{2,3}", s))

def to_full_language_name(val):
    if val is None:
        return ""
    s = str(val).strip()
    if s == "" or s.lower() == "none":
        return ""
    # אם זה לא נראה כמו קוד (כבר שם מלא?) — נשאיר כמו שהוא
    if not is_code_like(s):
        return s
    c = s.lower()
    # מפה ידנית קודם
    if c in MANUAL_MAP:
        return MANUAL_MAP[c]
    # pycountry: alpha-2
    try:
        lang = pycountry.languages.get(alpha_2=c)
        if lang and getattr(lang, "name", None):
            return lang.name
    except Exception:
        pass
    # pycountry: alpha-3
    try:
        lang = pycountry.languages.get(alpha_3=c)
        if lang and getattr(lang, "name", None):
            return lang.name
    except Exception:
        pass
    # נפילה חזרה — נשאיר כמו שהוא (לא נכתוב קוד ריק)
    return s

# המר רק ערכים שממש קוד/קוד מזוהה
df["language"] = df["language"].apply(to_full_language_name).fillna("").astype("string")

# 🔒 גיבוי לפני כתיבה
backup = CSV.with_suffix(".backup.csv")
df.to_csv(backup, index=False, encoding="utf-8-sig")
print(f"🛟 Backup written: {backup}")

# כתיבה חזרה
df.to_csv(CSV, index=False, encoding="utf-8-sig")
filled = (df["language"].str.len() > 0).sum()
print(f"✅ language הומר לשמות מלאים ונשמר. שורות עם שפה לא-ריקה: {filled} מתוך {len(df)}")

try:
    display(df.head(10)[["username","poi_name","language"]])
except Exception:
    pass


In [ ]:
# ============================================
# STEP 8 — POI Twitter statistics & summaries (portable, updated)
# ============================================

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, re, glob, math
import pandas as pd
from datetime import datetime
from pathlib import Path
from IPython.display import display

# ---------- איתור דינמי של תיקיית Iran ו-POIs ----------
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran')
if not BASE:
    BASE = '/content/drive/MyDrive/Iran'
    os.makedirs(BASE, exist_ok=True)
    print("⚠️ לא נמצאה תיקיית 'Iran' משותפת — נוצרה חדשה מקומית:", BASE)
else:
    print("📁 נמצאה תיקיית Iran:", BASE)

REL = "POIs"
POIs_DIR = os.path.join(BASE, REL)
os.makedirs(POIs_DIR, exist_ok=True)

# ---------- עזרה: מציאת קבצים + קריאה עמידה ----------
def newest(pattern: str):
    files = glob.glob(os.path.join(POIs_DIR, pattern))
    if not files:
        return None
    files.sort(key=lambda f: os.path.getmtime(f), reverse=True)
    return files[0]

def safe_read_csv(path, nrows=None):
    for enc in (None, "utf-8", "utf-8-sig"):
        try:
            return pd.read_csv(path, nrows=nrows, encoding=enc)
        except Exception:
            pass
    try:
        return pd.read_csv(path, nrows=nrows, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
    except Exception:
        return None

DATA_CSV   = newest("POI_twitter_users_data*.csv")
# שם חדש בהתאם לשינויים הקודמים
MANUAL_CSV = newest("Manual_Search_POIs*.csv")  # אופציונלי, להפקת קטגוריה

assert DATA_CSV is not None, "לא נמצא קובץ POI_twitter_users_data*.csv – יש להריץ את שלב 7 קודם."

print("📄 DATA :", DATA_CSV)
print("📄 MANUAL (optional):", MANUAL_CSV)

# ---------- טעינת הנתונים ----------
df = safe_read_csv(DATA_CSV)
if df is None:
    raise RuntimeError(f"לא הצלחתי לקרוא את הקובץ: {DATA_CSV}")

# נוודא עמודות יסוד – נוצרו בשלב 7
for col in ["username", "followers_count", "following_count", "statuses_count", "description / bio"]:
    if col not in df.columns:
        df[col] = None

# ---------- הוספת קטגוריה (אם יש Manual) ----------
HANDLE_RE = re.compile(r"^[A-Za-z0-9_]{1,15}$")

def extract_username(val):
    if pd.isna(val):
        return None
    s = str(val).strip()
    s = s.replace("https://","").replace("http://","").replace("www.","")
    m = re.search(r"(?:twitter\.com|x\.com)/([A-Za-z0-9_]{1,15})", s, re.I)
    if m: return m.group(1)
    m = re.search(r"@([A-Za-z0-9_]{1,15})", s)
    if m: return m.group(1)
    if HANDLE_RE.fullmatch(s): return s
    return None

category_col = None
if MANUAL_CSV:
    manual = safe_read_csv(MANUAL_CSV)
    if manual is not None and not manual.empty:
        # עמודת יוזר אפשרית: Twitter_username / username / ...
        cand_user_cols = [c for c in manual.columns if str(c).lower().strip() in
                          {"twitter_username","username","tw_username","twitter user","user"}]
        if not cand_user_cols:
            # ניסיון חילוץ מהעמודה עם קישור
            url_like = []
            for c in manual.columns:
                try:
                    sample = " ".join(map(str, manual[c].dropna().astype(str).head(100).tolist())).lower()
                except Exception:
                    sample = ""
                if "twitter" in str(c).lower() or "x.com/" in sample or "twitter.com/" in sample:
                    url_like.append(c)
            cand_user_cols = url_like[:1]

        if cand_user_cols:
            mu = manual.copy()
            mu["__user_join_key__"] = mu[cand_user_cols[0]].apply(extract_username).str.lower()

            # קטגוריה: קודם source_folder אם קיים, אחרת keyword, אחרת unknown
            if "source_folder" in mu.columns:
                mu["category"] = mu["source_folder"].astype(str).str.strip().str.lower()
            else:
                cand_cat = None
                for c in ["keyword", "category", "Категория", "קטגוריה"]:
                    if c in mu.columns:
                        cand_cat = c
                        break
                if cand_cat:
                    mu["category"] = (mu[cand_cat].astype(str).str.strip()
                                      .str.replace(r"\s+", "_", regex=True).str.lower())
                else:
                    mu["category"] = "unknown"

            df["__user_join_key__"] = df["username"].astype(str).str.lower()
            df = df.merge(mu[["__user_join_key__", "category"]], on="__user_join_key__", how="left")
            df.drop(columns=["__user_join_key__"], inplace=True, errors="ignore")
            category_col = "category"

if category_col is None:
    df["category"] = "unknown"
    category_col = "category"

# ---------- ניקוי טיפוסי עמודות מספריות ----------
for c in ["followers_count", "following_count", "statuses_count"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------- חישובי מאקרו ----------
num_pois_total = df["username"].nunique()

avg_followers = df["followers_count"].mean(skipna=True)
avg_following = df["following_count"].mean(skipna=True)
avg_posts     = df["statuses_count"].mean(skipna=True)

print("\n🌟 סיכום כללי:")
print(f"- מספר ה-POIs הכולל: {num_pois_total}")
print(f"- ממוצע followers : {avg_followers:,.0f}" if not math.isnan(avg_followers) else "- אין נתונים לעוקבים")
print(f"- ממוצע following : {avg_following:,.0f}" if not math.isnan(avg_following) else "- אין נתונים לנעקבים")
print(f"- ממוצע posts     : {avg_posts:,.0f}"     if not math.isnan(avg_posts)     else "- אין נתונים לפוסטים")

# ---------- טבלה 1: מאפיינים כמותיים לפי קטגוריה ----------
grp = (df.groupby("category", dropna=False)
         .agg(
             Num_POIs=("username","nunique"),
             Avg_Followers=("followers_count","mean"),
             Avg_Following=("following_count","mean"),
             Avg_Posts=("statuses_count","mean"),
          )
         .reset_index()
      )
for c in ["Avg_Followers","Avg_Following","Avg_Posts"]:
    grp[c] = grp[c].round(0)

# ---------- טבלה 2: Occupation מתוך הביו (כולל בריאות) ----------
occ_map = {
    r"\bminister|mp|parliament|politic|senator|governor|president|ambassador|diplomat|mayor|congress|knesset\b": "Politician",
    r"\bjournalist|reporter|editor|news|press\b": "Journalist",
    r"\bactor|actress|cinema|film|movie|director|producer|screenwriter\b": "Artist",
    r"\bsinger|vocal|music|musician|composer|band|rapper|songwriter\b": "Artist",
    r"\bfootball|soccer|fifa|goalkeeper|striker|midfielder|defender|athlete\b": "Athlete",
    r"\bscientist|researcher|professor|phd|chemist|physicist|biologist|engineer\b": "Scientist",
    r"\bwriter|author|poet|novelist\b": "Writer",
    r"\beconomist|economic\b": "Economist",
    r"\bayatollah|cleric|imam|rabbi|priest\b": "Religious_Leader",
    # Healthcare הרחבות:
    r"\bphysician|doctor|md|surgeon|cardiolog|oncolog|neurolog|pediatric|hospital|clinic|health\b": "Healthcare",
}

def infer_occ(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "Unknown"
    s = text.lower()
    for pat, label in occ_map.items():
        if re.search(pat, s):
            return label
    return "Other/Unknown"

df["Occupation"] = df["description / bio"].apply(infer_occ)

occ = (df.groupby("Occupation", dropna=False)
         .agg(Count=("username","nunique"))
         .reset_index()
      )
total = occ["Count"].sum() if len(occ) else 0
occ["Percentage (%)"] = (occ["Count"] / total * 100).round(1) if total else 0

# ---------- שמירה לקבצים ----------
stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
OUT_MASTER = os.path.join(POIs_DIR, "POI_twitter_users_statistics.csv")
OUT_CAT    = os.path.join(POIs_DIR, "POI_stats_by_category.csv")
OUT_OCC    = os.path.join(POIs_DIR, "POI_stats_by_occupation.csv")

summary_row = pd.DataFrame([{
    "metric":"overview",
    "num_pois_total": num_pois_total,
    "avg_followers": round(avg_followers,0) if not math.isnan(avg_followers) else None,
    "avg_following": round(avg_following,0) if not math.isnan(avg_following) else None,
    "avg_posts":     round(avg_posts,0)     if not math.isnan(avg_posts)     else None,
    "generated_at_utc": stamp
}])

# נשמור “מאסטר” + נצרף אליו את שתי הטבלאות
summary_row.to_csv(OUT_MASTER, index=False, encoding="utf-8-sig")
with open(OUT_MASTER, "a", encoding="utf-8-sig") as f:
    f.write("\n# --- by category ---\n")
    grp.to_csv(f, index=False)
    f.write("\n# --- by occupation ---\n")
    occ.to_csv(f, index=False)

# בנוסף נשמור כל טבלה בנפרד
grp.to_csv(OUT_CAT, index=False, encoding="utf-8-sig")
occ.to_csv(OUT_OCC, index=False, encoding="utf-8-sig")

print(f"\n✅ נשמרו הקבצים:")
print("  •", OUT_MASTER)
print("  •", OUT_CAT)
print("  •", OUT_OCC)

print("\n🔎 הצצה מהירה — טבלה 1 (לפי קטגוריה):")
display(grp.sort_values("Num_POIs", ascending=False).head(10))

print("\n🔎 הצצה מהירה — טבלה 2 (Occupation):")
display(occ.sort_values("Count", ascending=False))


In [ ]:
# ================================
# STEP 8 — Visuals only (no CSVs) — portable (UPDATED)
# ================================
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, pandas as pd, matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from IPython.display import display

# -------- Locate 'Iran/POIs' dynamically --------
def find_shared_folder(folder_name='Iran'):
    base_path = '/content/drive'
    for root, dirs, files in os.walk(base_path):
        for d in dirs:
            if d.lower() == folder_name.lower():
                return os.path.join(root, d)
    return None

BASE = find_shared_folder('Iran') or '/content/drive/MyDrive/Iran'
os.makedirs(BASE, exist_ok=True)
POIs_DIR = os.path.join(BASE, "POIs")
os.makedirs(POIs_DIR, exist_ok=True)

# -------- Ensure figures/stage8_visuals exists --------
FIG_DIR = os.path.join(POIs_DIR, "figures")
OUT_DIR = os.path.join(FIG_DIR, "stage8_visuals")
os.makedirs(OUT_DIR, exist_ok=True)
print("📁 Saving figures to:", OUT_DIR)

# -------- Helpers --------
def safe_read_csv(path):
    for enc in (None, "utf-8", "utf-8-sig"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path, engine="python", encoding="utf-8-sig", on_bad_lines="skip")

def thousands(x, pos):
    try:
        return f"{int(x):,}"
    except Exception:
        return x

def title_case(s):
    return str(s).replace("_", " ").title()

# -------- Load summary CSVs created in step 8 --------
cat_csv = os.path.join(POIs_DIR, "POI_stats_by_category.csv")     # category, Num_POIs, Avg_Followers, Avg_Following, Avg_Posts
occ_csv = os.path.join(POIs_DIR, "POI_stats_by_occupation.csv")   # Occupation, Count, Percentage (%)

assert os.path.exists(cat_csv), f"❌ חסר קובץ: {cat_csv} (הרץ את STEP 8 המלא קודם)"
assert os.path.exists(occ_csv), f"❌ חסר קובץ: {occ_csv} (הרץ את STEP 8 המלא קודם)"

df_cat = safe_read_csv(cat_csv)
df_occ = safe_read_csv(occ_csv)

# ניקוי שמות עמודות
df_cat.columns = [c.strip() for c in df_cat.columns]
df_occ.columns = [c.strip() for c in df_occ.columns]

# ודא עמודות חובה קיימות
for c in ["category", "Num_POIs", "Avg_Followers", "Avg_Following", "Avg_Posts"]:
    if c not in df_cat.columns:
        raise ValueError(f"עמודה חסרה ב-POI_stats_by_category.csv: {c}")

for c in ["Occupation", "Count"]:
    if c not in df_occ.columns:
        raise ValueError(f"עמודה חסרה ב-POI_stats_by_occupation.csv: {c}")
if "Percentage (%)" not in df_occ.columns:
    df_occ["Percentage (%)"] = (df_occ["Count"] / df_occ["Count"].sum() * 100) if df_occ["Count"].sum() else 0

# המרות ומיזוג ערכים חריגים
num_cols_cat = ["Num_POIs", "Avg_Followers", "Avg_Following", "Avg_Posts"]
df_cat[num_cols_cat] = df_cat[num_cols_cat].apply(pd.to_numeric, errors="coerce").fillna(0)

df_occ["Count"] = pd.to_numeric(df_occ["Count"], errors="coerce").fillna(0)
df_occ["Percentage (%)"] = pd.to_numeric(df_occ["Percentage (%)"], errors="coerce").fillna(0).clip(lower=0, upper=100)

# למנוע כפילות Other/Unknown ו-Unknown
df_occ["Occupation"] = df_occ["Occupation"].replace({"Other/Unknown": "Unknown"})
df_occ = (df_occ.groupby("Occupation", as_index=False)
                 .agg({"Count": "sum", "Percentage (%)": "sum"}))
df_occ["Percentage (%)"] = df_occ["Percentage (%)"].clip(upper=100)

# סדר להצגה יפה
df_cat = df_cat.sort_values("Num_POIs", ascending=True)
df_occ = df_occ.sort_values("Percentage (%)", ascending=True)

# -------- 1) ברים: מספר ה-POIs לכל קטגוריה --------
plt.figure(figsize=(11, 6))
plt.barh([title_case(c) for c in df_cat["category"]], df_cat["Num_POIs"])
plt.gca().xaxis.set_major_formatter(FuncFormatter(thousands))
plt.xlabel("Number of POIs")
plt.title("POIs per Category")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pois_per_category.png"), dpi=150)
plt.savefig(os.path.join(OUT_DIR, "pois_per_category.svg"))
plt.show()

# -------- 2) ברים: ממוצע Followers לפי קטגוריה --------
plt.figure(figsize=(11, 6))
plt.barh([title_case(c) for c in df_cat["category"]], df_cat["Avg_Followers"])
plt.gca().xaxis.set_major_formatter(FuncFormatter(thousands))
plt.xlabel("Average Followers")
plt.title("Average Followers by Category")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "avg_followers_by_category.png"), dpi=150)
plt.savefig(os.path.join(OUT_DIR, "avg_followers_by_category.svg"))
plt.show()

# -------- 3) ברים: ממוצע Following לפי קטגוריה --------
plt.figure(figsize=(11, 6))
plt.barh([title_case(c) for c in df_cat["category"]], df_cat["Avg_Following"])
plt.gca().xaxis.set_major_formatter(FuncFormatter(thousands))
plt.xlabel("Average Following")
plt.title("Average Following by Category")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "avg_following_by_category.png"), dpi=150)
plt.savefig(os.path.join(OUT_DIR, "avg_following_by_category.svg"))
plt.show()

# -------- 4) ברים: התפלגות Occupation באחוזים --------
plt.figure(figsize=(10, 6))
plt.barh(df_occ["Occupation"], df_occ["Percentage (%)"])
plt.xlabel("Percentage (%)")
plt.title("Occupation Distribution of POIs")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "occupation_distribution.png"), dpi=150)
plt.savefig(os.path.join(OUT_DIR, "occupation_distribution.svg"))
plt.show()

print("✅ Done. Visuals saved in:", OUT_DIR)


**API Twitter**

In [ ]:
pip install selenium webdriver-manager pandas

In [ ]:
!apt-get update -y
!apt-get install -y chromium-browser || apt-get install -y chromium
!apt-get install -y chromium-chromedriver || apt-get install -y chromium-driver


In [ ]:
# === דיאגנוסטיקה + ריענון התקנות ===
!apt-get update -y
# נתקין גם chromium וגם שני וריאנטים של chromedriver (לפי הדיסטרו של Colab)
!apt-get install -y chromium-browser || apt-get install -y chromium
!apt-get install -y chromium-chromedriver || apt-get install -y chromium-driver

# בדיקת גרסאות ונתיבים
!which -a chromium chromium-browser || true
!which -a chromedriver || true
!chromium --version || chromium-browser --version
!chromedriver --version
!ls -l /usr/bin/chromedriver || true
!ls -l /usr/lib/chromium-browser/chromedriver || true


In [ ]:
!pip install selenium


In [ ]:
# ============================================
# Step 9 — Followers/Following for POIs (Colab, Selenium)
# ============================================

# ---------- 🔧 CONFIG ----------
BASE_DRIVE_DIR = "/content/drive/MyDrive/Iran/POIs"            # 🔧 שורש הפרויקט בדרייב
USERS_CSV      = f"{BASE_DRIVE_DIR}/POI_twitter_users_data.csv" # חייב עמודה 'username'
OUT_DIR        = f"{BASE_DRIVE_DIR}/Candidates"
COOKIES_JSON   = f"{BASE_DRIVE_DIR}/x_cookies.json"            # 🔧 קוקיז JSON מחשבון X המחובר

# מהיר לבדיקה:
SAMPLE_SIZE  = None        # ← עודכן ל-5
DO_FOLLOWERS = True
DO_FOLLOWING = True
MAX_PER_LIST = 80
SCROLL_STEPS = 15
OPEN_PAUSE   = 1.6
SCROLL_PAUSE = 0.7

# אפשר לבחור ידנית במקום לקחת מה-CSV:
MANUAL_USERS = []  # לדוגמה: ["amir_jadidi", "OAradwinwin", "Golshifteh"]

# ---------- MOUNT ----------
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ---------- IMPORTS ----------
import os, json, time, random, re, pandas as pd, shutil, stat
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# ---------- FOLDERS ----------
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ---------- INIT CHROMIUM ----------
def _which(cmds):
    import shutil as _sh
    for c in cmds:
        p = _sh.which(c)
        if p:
            return p
    return None
def _chmod_x(path):
    try:
        mode = os.stat(path).st_mode
        os.chmod(path, mode | stat.S_IEXEC)
    except Exception:
        pass

CHROME_BIN = _which(["chromium-browser","chromium","google-chrome","chrome"])
CHROMEDRIVER_CANDIDATES = [
    _which(["chromedriver"]),
    "/usr/bin/chromedriver",
    "/usr/lib/chromium-browser/chromedriver",
]
assert CHROME_BIN, "Chromium/Chrome לא נמצא. ודא שהרצת את תא ההתקנה."

opts = Options()
opts.binary_location = CHROME_BIN
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-gpu")
opts.add_argument("--window-size=1280,2000")
opts.add_argument("--lang=en-US,en;q=0.9")

driver = None
for drv in [p for p in CHROMEDRIVER_CANDIDATES if p and os.path.exists(p)]:
    try:
        _chmod_x(drv)
        service = Service(drv)
        driver  = webdriver.Chrome(service=service, options=opts)
        break
    except Exception as e:
        print("chromedriver try failed:", e)
        driver = None
if driver is None:
    driver = webdriver.Chrome(options=opts)  # Selenium Manager fallback
print("✅ WebDriver is up.")

# ---------- COOKIES ----------
def load_cookies(cookies_path: str):
    driver.get("https://x.com/")
    time.sleep(1.5)
    with open(cookies_path, "r", encoding="utf-8") as f:
        cookies = json.load(f)
    injected = 0
    for c in cookies:
        try:
            c_fixed = {
                "name": c.get("name"),
                "value": c.get("value"),
                "domain": c.get("domain", ".x.com"),
                "path": c.get("path", "/"),
                "secure": bool(c.get("secure", True)),
                "httpOnly": bool(c.get("httpOnly", False))
            }
            if "expiry" in c:           c_fixed["expiry"] = int(c["expiry"])
            elif "expirationDate" in c: c_fixed["expiry"] = int(c["expirationDate"])
            driver.add_cookie(c_fixed); injected += 1
        except Exception:
            pass
    driver.refresh(); time.sleep(2.0)
    print(f"Injected {injected} cookies.")

# ---------- SCRAPE HELPERS ----------
USER_HREF_RE = re.compile(r"^/[A-Za-z0-9_]{1,15}$")

def _extract_usernames_from_page():
    anchors = driver.find_elements(By.XPATH, "//a[@href]")
    names = set()
    for a in anchors:
        try:
            href = a.get_attribute("href") or ""
            path  = "/" + href.split("://",1)[-1].split("/",1)[-1]
            first = "/" + path.strip("/").split("/",1)[0]
            if USER_HREF_RE.fullmatch(first):
                cand = first.strip("/")
                if cand.lower() in {"home","i","explore","notifications","messages","settings"}:
                    continue
                names.add(cand)
        except:
            pass
    return list(names)

def _smooth_scroll(steps):
    last_h = driver.execute_script("return document.body.scrollHeight")
    for _ in range(steps):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE + random.uniform(0.15, 0.35))
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            time.sleep(0.5)
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break
        last_h = new_h

def _collect_list(user, tab):
    url = f"https://x.com/{user}/{tab}"
    driver.get(url); time.sleep(OPEN_PAUSE + random.uniform(0.1, 0.3))
    got = set()
    for _ in range(SCROLL_STEPS):
        got.update(_extract_usernames_from_page())
        if len(got) >= MAX_PER_LIST:
            break
        _smooth_scroll(1)
    got.discard(user)
    return list(got)[:MAX_PER_LIST]

# ---------- Full name helper (לדרישת ה-README) ----------
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def _get_full_name(user):
    """מחזיר את שם התצוגה (Full Name) מפרופיל המשתמש, או None אם לא נמצא."""
    try:
        driver.get(f"https://x.com/{user}")
        time.sleep(OPEN_PAUSE)
        el = WebDriverWait(driver, 6).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="UserName"]'))
        )
        txt = el.text or ""
        return txt.split("\n")[0].strip() if txt else None
    except Exception:
        return None

# ---------- LOAD USERS ----------
df = pd.read_csv(USERS_CSV)
assert "username" in df.columns, "בקובץ המשתמשים חסרה עמודה 'username'"
users_all = [str(u).strip().lstrip("@") for u in df["username"].dropna()]
users_all = list(dict.fromkeys([u for u in users_all if u]))

if MANUAL_USERS:
    users = [u.strip().lstrip("@") for u in MANUAL_USERS] if SAMPLE_SIZE in (None, 0) else [u.strip().lstrip("@") for u in MANUAL_USERS][:SAMPLE_SIZE]
else:
    users = users_all if SAMPLE_SIZE in (None, 0) else users_all[:SAMPLE_SIZE]

print(f"Processing {len(users)} users:", users[:10], "..." if len(users) > 10 else "")


# ---------- RUN ----------
load_cookies(COOKIES_JSON)  # חשוב: קוקיז עדכניים מהחשבון שלך
connections, stats = [], []

for uname in users:
    print(f"\n➡️ {uname}")
    errs = []
    followers, following = [], []

    if DO_FOLLOWERS:
        try:
            followers = _collect_list(uname, "followers")
        except Exception as e:
            errs.append(f"followers_error:{e}")

    if DO_FOLLOWING:
        try:
            following = _collect_list(uname, "following")
        except Exception as e:
            errs.append(f"following_error:{e}")

    for x in followers:
        connections.append((uname, x, "follower"))
    for x in following:
        connections.append((uname, x, "following"))

    full_name = _get_full_name(uname)  # ← חדש: שם תצוגה לסטטיסטיקות
    stats.append({
        "Twitter_username": uname,
        "Full_Name": full_name,                       # ← חדש
        "Followers_Collected": len(followers),
        "Following_Collected": len(following),
        "Error": "; ".join(errs) if errs else None
    })

# ---------- SAVE (בדיוק לפי הדרישות של סעיף 9) ----------
CONN_CSV = os.path.join(OUT_DIR, "POIs_candidate_connections.csv")
STAT_CSV = os.path.join(OUT_DIR, "Candidates_statistics.csv")

# מחיקה אוטומטית כדי שתראה עדכון מיידי
for path in [CONN_CSV, STAT_CSV]:
    if os.path.exists(path):
        os.remove(path)
        print(f"🧹 Deleted old file: {path}")

(pd.DataFrame(connections, columns=["target_username","other_username","type"])
   .drop_duplicates()
   .to_csv(CONN_CSV, index=False, encoding="utf-8-sig"))

(pd.DataFrame(stats)[["Twitter_username","Full_Name","Followers_Collected","Following_Collected","Error"]]
   .to_csv(STAT_CSV, index=False, encoding="utf-8-sig"))

print("✅ Done")
print("connections:", CONN_CSV, f"rows={len(connections)}")
print("stats:", STAT_CSV)


In [ ]:
!pip -q install playwright tqdm pandas
!playwright install --with-deps chromium


In [ ]:
# ============================================
# Step 10 — Candidates user metadata (built on your twitter_selenium scraper)
# Output: /content/drive/MyDrive/Iran/POIs/Candidates/Candidates_user_data.csv
# ============================================

# --- אם צריך התקנות בסיס (בטל הערה והריץ פעם אחת) ---
# !apt-get update -y && apt-get install -y chromium-browser chromium-chromedriver
# !pip install -U selenium tqdm pandas

# ---------- CONFIG ----------
BASE_DRIVE_DIR = "/content/drive/MyDrive/Iran/POIs"
CAND_DIR       = f"{BASE_DRIVE_DIR}/Candidates"

USERS_CSV      = f"{BASE_DRIVE_DIR}/POI_twitter_users_data.csv"     # קלט: ה-POIs המקוריים (עמודה: username)
CONN_CSV       = f"{CAND_DIR}/POIs_candidate_connections.csv"       # קלט: שלב 9 (עמודות: target_username, other_username)
OUT_CSV        = f"{CAND_DIR}/Candidates_user_data.csv"             # פלט: שלב 10

# 🔹 קובץ ה-scraper שלך; אם שמור ליד הנוטבוק, עדכן PATH מתאים:
SCRAPER_PATH = "/content/drive/MyDrive/Iran/POIs/tools/twitter_selenium.py"


# שליטה בהרצה
SAMPLE_LIMIT      = 4000     # ← בדיקת עשן על 10; שנה ל-None כדי לרוץ על כולם
BATCH_SIZE        = 40      # כמה משתמשים בכל מנה
RETRIES_PER_USER  = 2       # ניסיונות לכל משתמש
OPEN_PAUSE        = 1.4     # שניות המתנה אחרי פתיחת פרופיל

# מיפוי עמודות: מהסקרולר שלך אל הסכימה התקנית של הפרויקט
COLUMNS = [
    "username","Full_Name","Bio","Location","External_URL",
    "Followers_Count","Following_Count","Is_Verified","Is_Protected",
    "Joined_Date","Profile_URL","Profile_Image"
]

# ---------- MOUNT ----------
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ---------- IMPORTS ----------
import os, json, time, shutil, pandas as pd, sys, importlib.util, re
from pathlib import Path
from tqdm.auto import tqdm

Path(CAND_DIR).mkdir(parents=True, exist_ok=True)

# ---------- LOAD YOUR SCRAPER MODULE ----------
def load_scraper(scraper_path: str):
    if not os.path.exists(scraper_path):
        raise FileNotFoundError(f"twitter_selenium.py not found at: {scraper_path}")
    spec = importlib.util.spec_from_file_location("twitter_selenium", scraper_path)
    mod  = importlib.util.module_from_spec(spec)
    sys.modules["twitter_selenium"] = mod
    spec.loader.exec_module(mod)
    return mod

ts = load_scraper(SCRAPER_PATH)  # מכיל scrape_twitter_profile(username)

# ---------- HELPERS ----------
def normalize_row_from_scraper(d: dict):
    """
    ממפה את פלט הסקרולר שלך לשדות של הפרויקט.
    scraper keys (your file): user_name, name, bio, location, url, joined_date,
                              followers, following, verified, profile_image, profile_url
    """
    return {
        "username":       (d.get("user_name") or "").lstrip("@"),
        "Full_Name":      d.get("name"),
        "Bio":            d.get("bio"),
        "Location":       d.get("location"),
        "External_URL":   d.get("url"),
        "Followers_Count":d.get("followers"),
        "Following_Count":d.get("following"),
        "Is_Verified":    bool(d.get("verified")),
        "Is_Protected":   None,  # הסקרולר שלך לא מחלץ; אפשר להשאיר None (לא נדרש בסעיף)
        "Joined_Date":    d.get("joined_date"),
        "Profile_URL":    d.get("profile_url"),
        "Profile_Image":  d.get("profile_image")
    }

# ---------- BUILD USER SET (שלב 9 + המקור, ללא כפילויות) ----------
user_set = set()

if os.path.exists(CONN_CSV):
    df_conn = pd.read_csv(CONN_CSV)
    if "target_username" in df_conn.columns:
        user_set.update(df_conn["target_username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())
    if "other_username" in df_conn.columns:
        user_set.update(df_conn["other_username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())

if os.path.exists(USERS_CSV):
    df_users = pd.read_csv(USERS_CSV)
    if "username" in df_users.columns:
        user_set.update(df_users["username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())

users_all = [u for u in dict.fromkeys([u for u in user_set if u])]  # ייחוד + שמירת סדר
if SAMPLE_LIMIT:
    users_all = users_all[:SAMPLE_LIMIT]

print(f"Users to enrich: {len(users_all)} → {users_all[:5]}")

# ---------- HEADER BOOTSTRAP (מבטיח כותרות) ----------
if not os.path.exists(OUT_CSV):
    pd.DataFrame(columns=COLUMNS).to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# ---------- RESUME (דלג על מי שכבר נשמר) ----------
done = set()
try:
    prev = pd.read_csv(OUT_CSV, usecols=["username"])
    done = set(prev["username"].dropna().astype(str))
    print(f"↻ Resume: skipping {len(done)} already saved.")
except Exception as e:
    print("Resume read issue:", e)

# ---------- RUN (במנות) ----------
from math import ceil
total   = len(users_all)
batches = ceil(total / BATCH_SIZE)
idx = 0

for b in range(batches):
    chunk = [u for u in users_all[idx: idx+BATCH_SIZE] if u not in done]
    idx += BATCH_SIZE
    if not chunk:
        continue

    with tqdm(total=len(chunk), desc=f"Batch {b+1}/{batches}", unit="user") as pbar:
        for uname in chunk:
            row = None
            # ניסיונות חוזרים לכל משתמש (הסקרולר יפתח/יסגור כרום בעצמו)
            for attempt in range(RETRIES_PER_USER + 1):
                try:
                    raw = ts.scrape_twitter_profile(uname)   # ⚠️ מתוך twitter_selenium.py שלך
                    row = normalize_row_from_scraper(raw)
                    break
                except Exception as e:
                    if attempt < RETRIES_PER_USER:
                        time.sleep(0.8)
                    else:
                        # במקרה כשל — נשמור שורה "ריקה" עם השם וה-URL בלבד, כדי לא לחסום התקדמות
                        row = {
                            "username": uname, "Full_Name": None, "Bio": None, "Location": None,
                            "External_URL": None, "Followers_Count": None, "Following_Count": None,
                            "Is_Verified": None, "Is_Protected": None, "Joined_Date": None,
                            "Profile_URL": f"https://twitter.com/{uname}", "Profile_Image": None
                        }

            pd.DataFrame([row], columns=COLUMNS).to_csv(
                OUT_CSV, mode="a", header=False, index=False, encoding="utf-8-sig"
            )
            done.add(uname)
            pbar.update(1)

# ---------- דה-דופליקציה (ליתר ביטחון) ----------
try:
    df_out = pd.read_csv(OUT_CSV)
    df_out = df_out.drop_duplicates(subset=["username"], keep="first")
    df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print("✅ Saved:", OUT_CSV)
    print(df_out.head())
except Exception as e:
    print("Compact error:", e)


In [ ]:
ls /content/drive/MyDrive/Iran/POIs/tools
